- google results
- maps link
- maps reviews number
- maps reviews stars
- coordinates

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
import time

In [39]:
data = pd.read_csv("fulldata_draft.csv", index_col=0)

In [40]:
data

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,TA_rating,TA_reviews,TA_url,wiki_coordinates,WKT_LNG_LAT,District,Neighborhoodcombination,Neighborhood,UNESCO,Monuments
0,AFAS Live,Monument of Gebouw,<WikipediaPage 'AFAS Live'>,https://nl.wikipedia.org/wiki/AFAS_Live,124539.0,0.249267,105429.0,1.123401e-02,45,0.029333,...,4.0,260.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.31222222, 4.94416667)",POINT (4.94416667 52.31222222),Zuidoost,Amstel III/Bullewijk,Hoofdcentrum Zuidoost,0,0
1,Albert Cuypmarkt,Markt,<WikipediaPage 'Albert Cuypmarkt'>,https://nl.wikipedia.org/wiki/Albert_Cuypmarkt,58069.0,0.116201,110365.0,1.175996e-02,70,0.046000,...,4.0,2428.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35597222, 4.89538889)",POINT (4.89538889 52.35597222),Zuid,Oude Pijp,Sarphatiparkbuurt,0,3
2,Allard Pierson Museum,Museum,<WikipediaPage 'Allard Pierson Museum'>,https://nl.wikipedia.org/wiki/Allard_Pierson_M...,33905.0,0.067828,27502.0,2.930481e-03,109,0.072000,...,3.5,172.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36861111, 4.89333333)",POINT (4.89333333 52.36861111),Centrum,Burgwallen-Oude Zijde,BG-terrein e.o.,0,44
3,Amstelkerk,Monument of Gebouw,<WikipediaPage 'Amstelkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amstelkerk_(Amst...,13384.0,0.026747,2.0,2.131104e-07,52,0.034000,...,4.5,6.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36222222, 4.89638889)",POINT (4.89638889 52.36222222),Centrum,Grachtengordel-Zuid,Amstelveldbuurt,56,0
4,Amstelveld,Markt,<WikipediaPage 'Amstelveld'>,https://nl.wikipedia.org/wiki/Amstelveld,20987.0,0.041968,53163.0,5.664794e-03,47,0.030667,...,4.0,12.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36222222, 4.8975)",POINT (4.8975 52.36222222),Centrum,Grachtengordel-Zuid,Amstelveldbuurt,43,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,Siegerpark,Park of recreatiegebied,<WikipediaPage 'Siegerpark'>,https://nl.wikipedia.org/wiki/Siegerpark,7495.0,0.014958,150247.0,1.600960e-02,26,0.016667,...,4.0,6.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.34068889, 4.81939444)",POINT (4.81939444 52.34068889),Nieuw-West,Sloter-/Riekerpolder,Sloterweg e.o.,0,0
169,Noorderpark,Park of recreatiegebied,<WikipediaPage 'Noorderpark (Amsterdam)'>,https://nl.wikipedia.org/wiki/Noorderpark_(Ams...,8852.0,0.017675,0.0,0.000000e+00,37,0.024000,...,3.5,8.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.39472222, 4.91944444)",POINT (4.91944444 52.39472222),Noord,Volewijck,Bloemenbuurt Zuid,0,1
170,Nieuwmarkt,Markt,<WikipediaPage 'Nieuwmarkt (Amsterdam)'>,https://nl.wikipedia.org/wiki/Nieuwmarkt_(Amst...,43629.0,0.087294,58019.0,6.182226e-03,137,0.090667,...,4.0,333.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37257306, 4.90050316)",POINT (4.90050316 52.37257306),Centrum,Nieuwmarkt/Lastage,Nieuwmarkt,0,73
171,Amsterdamse Poort,Markt,<WikipediaPage 'Amsterdamse Poort (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amsterdamse_Poor...,17723.0,0.035433,0.0,0.000000e+00,54,0.035333,...,4.0,10.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.31388889, 4.9525)",POINT (4.9525 52.31388889),Zuidoost,"Bijlmer Centrum (D,F,H)",Amsterdamse Poort,0,0


In [44]:
data["name"] = data["name"].apply(lambda x: x.rstrip())

In [45]:
list(data["name"])

['AFAS Live',
 'Albert Cuypmarkt',
 'Allard Pierson Museum',
 'Amstelkerk',
 'Amstelveld',
 'Johan Cruijff ArenA',
 'Amsterdam Museum',
 'Anne Frank Huis',
 'Artis',
 'Basiliek van de Heilige Nicolaas',
 'Begijnhof',
 'Bijbels Museum',
 'Bimhuis',
 'Blauwbrug',
 'Bloemenmarkt',
 'Dam',
 'DeLaMar',
 'De Duif',
 'De Kleine Komedie',
 'De Krijtberg',
 'De Papegaai',
 'De Wallen',
 'Eye Filmmuseum',
 'Engelse Hervormde Kerk',
 'He Hua Tempel',
 'Hermitage Amsterdam',
 'Homomonument',
 'Kalverpassage',
 'Kleinste huis van Amsterdam',
 'Koninklijk Concertgebouw',
 'Koninklijk Theater Carré',
 'Leidseplein',
 'Lloyd Hotel',
 'Magere Brug',
 'Magna Plaza',
 'Melkweg',
 'Montelbaanstoren',
 'Munttoren',
 "Museum Ons' Lieve Heer op Solder",
 'Museum Van Loon',
 'Museumplein',
 "Muziekgebouw aan 't IJ",
 'NEMO Science Museum',
 'Nieuwe Kerk',
 'Noorderkerk',
 'Noordermarkt',
 'Olympisch Stadion',
 'Oost-Indisch Huis',
 'Oosterkerk',
 'Oude Kerk',
 'Paleis op de Dam',
 'Paradiso',
 'Posthoornkerk'

In [33]:
d = data.head(10)

In [50]:
d

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,TA_rating,TA_reviews,TA_url,wiki_coordinates,WKT_LNG_LAT,District,Neighborhoodcombination,Neighborhood,UNESCO,Monuments
0,AFAS Live,Monument of Gebouw,<WikipediaPage 'AFAS Live'>,https://nl.wikipedia.org/wiki/AFAS_Live,124539.0,0.249267,105429.0,1.123401e-02,45,0.029333,...,4.0,260.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.31222222, 4.94416667)",POINT (4.94416667 52.31222222),Zuidoost,Amstel III/Bullewijk,Hoofdcentrum Zuidoost,0,0
1,Albert Cuypmarkt,Markt,<WikipediaPage 'Albert Cuypmarkt'>,https://nl.wikipedia.org/wiki/Albert_Cuypmarkt,58069.0,0.116201,110365.0,1.175996e-02,70,0.046000,...,4.0,2428.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35597222, 4.89538889)",POINT (4.89538889 52.35597222),Zuid,Oude Pijp,Sarphatiparkbuurt,0,3
2,Allard Pierson Museum,Museum,<WikipediaPage 'Allard Pierson Museum'>,https://nl.wikipedia.org/wiki/Allard_Pierson_M...,33905.0,0.067828,27502.0,2.930481e-03,109,0.072000,...,3.5,172.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36861111, 4.89333333)",POINT (4.89333333 52.36861111),Centrum,Burgwallen-Oude Zijde,BG-terrein e.o.,0,44
3,Amstelkerk,Monument of Gebouw,<WikipediaPage 'Amstelkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amstelkerk_(Amst...,13384.0,0.026747,2.0,2.131104e-07,52,0.034000,...,4.5,6.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36222222, 4.89638889)",POINT (4.89638889 52.36222222),Centrum,Grachtengordel-Zuid,Amstelveldbuurt,56,0
4,Amstelveld,Markt,<WikipediaPage 'Amstelveld'>,https://nl.wikipedia.org/wiki/Amstelveld,20987.0,0.041968,53163.0,5.664794e-03,47,0.030667,...,4.0,12.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36222222, 4.8975)",POINT (4.8975 52.36222222),Centrum,Grachtengordel-Zuid,Amstelveldbuurt,43,0
5,Johan Cruijff ArenA,Monument of Gebouw,<WikipediaPage 'Johan Cruijff ArenA'>,https://nl.wikipedia.org/wiki/Johan_Cruijff_ArenA,184052.0,0.368405,433884.0,4.623260e-02,1501,1.000000,...,4.5,1365.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.31416667, 4.94194444)",POINT (4.94194444 52.31416667),Zuidoost,Amstel III/Bullewijk,Hoofdcentrum Zuidoost,0,0
6,Amsterdam Museum,Museum,<WikipediaPage 'Amsterdam Museum'>,https://nl.wikipedia.org/wiki/Amsterdam_Museum,53772.0,0.107599,91050.0,9.701851e-03,277,0.184000,...,4.0,2206.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37027778, 4.89083333)",POINT (4.89083333 52.37027778),Centrum,Burgwallen-Nieuwe Zijde,Begijnhofbuurt,0,96
7,Anne Frank Huis,Monument of Gebouw,<WikipediaPage 'Anne Frank Huis'>,https://nl.wikipedia.org/wiki/Anne_Frank_Huis,177070.0,0.354428,1121274.0,1.194776e-01,532,0.354000,...,4.5,62465.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37515278, 4.88413056)",POINT (4.88413056 52.37515278),Centrum,Grachtengordel-West,Leliegracht e.o.,51,0
8,Artis,Monument of Gebouw,<WikipediaPage 'Artis'>,https://nl.wikipedia.org/wiki/Artis,169659.0,0.339592,1406932.0,1.499159e-01,771,0.513333,...,4.5,7069.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36611111, 4.91583333)",POINT (4.91583333 52.36611111),Centrum,Weesperbuurt/Plantage,Plantage,0,12
9,Basiliek van de Heilige Nicolaas,Monument of Gebouw,<WikipediaPage 'Basiliek van de Heilige Nicola...,https://nl.wikipedia.org/wiki/Basiliek_van_de_...,48705.0,0.097456,14751.0,1.571796e-03,170,0.112667,...,4.5,493.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37652778, 4.90111111)",POINT (4.90111111 52.37652778),Centrum,Burgwallen-Oude Zijde,Kop Zeedijk,0,67


In [18]:
def scrape_google(monument):
    s = monument.replace(" ", "+")    
    url = 'http://www.google.com/search?q=' + s + "+amsterdam"
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36"}
    page = requests.get(url, headers=headers)        
    soup = BeautifulSoup(page.content, 'html.parser')
    
    text1 = soup.find("div", {"id": "result-stats"}).find(text=True, recursive=False)
    google_results_num = int(''.join([num for num in text1 if num.isdigit()]))    
    
    try:
        r1 = re.findall("\d+\.\d+", str(soup.find_all("a", {"data-async-trigger":"reviewDialog"})[0].find('span')))[0]
        reviews = int(r1.replace('.', ''))
    except IndexError:
        reviews = np.nan
    
    
    try:
        r2 = re.findall("\d+\,\d+", soup.find("span", {"class": "z3HNkc"})["aria-label"])
        stars = float(r2[0].replace(',', '.'))
    except TypeError:
        stars = np.nan
    
    
    coord = soup.find_all("a", {"data-url":True})
    try:
        continuation = coord[0]["data-url"]
        link_maps = "https://www.google.com/" + continuation
    except IndexError:
        link_maps = np.nan
    
    
    try:
        r = re.findall("\d+\.\d+",link_maps)
        try:
            lat = float(r[0])
            long = float(r[1])
            coordinates = (lat, long)
        except IndexError:
            coordinates = np.nan
    except TypeError:
        coordinates = np.nan

    
    return google_results_num, reviews, stars, link_maps, coordinates

In [52]:
z = d[d["name"] == "Anne Frank Huis"]
z

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,TA_rating,TA_reviews,TA_url,wiki_coordinates,WKT_LNG_LAT,District,Neighborhoodcombination,Neighborhood,UNESCO,Monuments
7,Anne Frank Huis,Monument of Gebouw,<WikipediaPage 'Anne Frank Huis'>,https://nl.wikipedia.org/wiki/Anne_Frank_Huis,177070.0,0.354428,1121274.0,0.119478,532,0.354,...,4.5,62465.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37515278, 4.88413056)",POINT (4.88413056 52.37515278),Centrum,Grachtengordel-West,Leliegracht e.o.,51,0


In [53]:
z["google"] = z["name"].apply(lambda a: scrape_google(a))

<ipython-input-53-1a902e036040>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z["google"] = z["name"].apply(lambda a: scrape_google(a))


In [56]:
z["a1"] = z["google"].apply(lambda a: a[0])
z["a2"] = z["google"].apply(lambda a: a[1])
z["a3"] = z["google"].apply(lambda a: a[2])
z["a4"] = z["google"].apply(lambda a: a[3])
z["a5"] = z["google"].apply(lambda a: a[4])
z

<ipython-input-56-acf2bd571f88>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z["a1"] = z["google"].apply(lambda a: a[0])
<ipython-input-56-acf2bd571f88>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z["a2"] = z["google"].apply(lambda a: a[1])
<ipython-input-56-acf2bd571f88>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google,a1,a2,a3,a4,a5
7,Anne Frank Huis,Monument of Gebouw,<WikipediaPage 'Anne Frank Huis'>,https://nl.wikipedia.org/wiki/Anne_Frank_Huis,177070.0,0.354428,1121274.0,0.119478,532,0.354,...,Grachtengordel-West,Leliegracht e.o.,51,0,"(3090000, 51090, 4.5, https://www.google.com//...",3090000,51090,4.5,https://www.google.com//maps/place/Anne+Frank+...,"(52.3752182, 4.8839765)"


In [63]:
d1 = data[:20]
d2 = data[20:40]
d3 = data[40:60]
d4 = data[60:80]
d5 = data[80:100]
d6 = data[100:120]
d7 = data[120:140]
d8 = data[140:160]
d9 = data[160:]


In [64]:
d1["google"] = d1["name"].apply(lambda a: scrape_google(a))

<ipython-input-64-ae8ce7750ff7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d1["google"] = d1["name"].apply(lambda a: scrape_google(a))


In [65]:
d1

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,TA_reviews,TA_url,wiki_coordinates,WKT_LNG_LAT,District,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google
0,AFAS Live,Monument of Gebouw,<WikipediaPage 'AFAS Live'>,https://nl.wikipedia.org/wiki/AFAS_Live,124539.0,0.249267,105429.0,1.123401e-02,45,0.029333,...,260.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.31222222, 4.94416667)",POINT (4.94416667 52.31222222),Zuidoost,Amstel III/Bullewijk,Hoofdcentrum Zuidoost,0,0,"(2070000, 9285, 4.3, https://www.google.com//m..."
1,Albert Cuypmarkt,Markt,<WikipediaPage 'Albert Cuypmarkt'>,https://nl.wikipedia.org/wiki/Albert_Cuypmarkt,58069.0,0.116201,110365.0,1.175996e-02,70,0.046000,...,2428.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35597222, 4.89538889)",POINT (4.89538889 52.35597222),Zuid,Oude Pijp,Sarphatiparkbuurt,0,3,"(109000, 25906, 4.3, https://www.google.com//m..."
2,Allard Pierson Museum,Museum,<WikipediaPage 'Allard Pierson Museum'>,https://nl.wikipedia.org/wiki/Allard_Pierson_M...,33905.0,0.067828,27502.0,2.930481e-03,109,0.072000,...,172.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36861111, 4.89333333)",POINT (4.89333333 52.36861111),Centrum,Burgwallen-Oude Zijde,BG-terrein e.o.,0,44,"(213000, nan, nan, https://www.google.com//map..."
3,Amstelkerk,Monument of Gebouw,<WikipediaPage 'Amstelkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amstelkerk_(Amst...,13384.0,0.026747,2.0,2.131104e-07,52,0.034000,...,6.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36222222, 4.89638889)",POINT (4.89638889 52.36222222),Centrum,Grachtengordel-Zuid,Amstelveldbuurt,56,0,"(71300, nan, 4.3, https://www.google.com//maps..."
4,Amstelveld,Markt,<WikipediaPage 'Amstelveld'>,https://nl.wikipedia.org/wiki/Amstelveld,20987.0,0.041968,53163.0,5.664794e-03,47,0.030667,...,12.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36222222, 4.8975)",POINT (4.8975 52.36222222),Centrum,Grachtengordel-Zuid,Amstelveldbuurt,43,0,"(54800, nan, nan, https://www.google.com//maps..."
5,Johan Cruijff ArenA,Monument of Gebouw,<WikipediaPage 'Johan Cruijff ArenA'>,https://nl.wikipedia.org/wiki/Johan_Cruijff_ArenA,184052.0,0.368405,433884.0,4.623260e-02,1501,1.000000,...,1365.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.31416667, 4.94194444)",POINT (4.94194444 52.31416667),Zuidoost,Amstel III/Bullewijk,Hoofdcentrum Zuidoost,0,0,"(1630000, 28005, 4.4, https://www.google.com//..."
6,Amsterdam Museum,Museum,<WikipediaPage 'Amsterdam Museum'>,https://nl.wikipedia.org/wiki/Amsterdam_Museum,53772.0,0.107599,91050.0,9.701851e-03,277,0.184000,...,2206.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37027778, 4.89083333)",POINT (4.89083333 52.37027778),Centrum,Burgwallen-Nieuwe Zijde,Begijnhofbuurt,0,96,"(97100000, 5328, 4.3, https://www.google.com//..."
7,Anne Frank Huis,Monument of Gebouw,<WikipediaPage 'Anne Frank Huis'>,https://nl.wikipedia.org/wiki/Anne_Frank_Huis,177070.0,0.354428,1121274.0,1.194776e-01,532,0.354000,...,62465.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37515278, 4.88413056)",POINT (4.88413056 52.37515278),Centrum,Grachtengordel-West,Leliegracht e.o.,51,0,"(3090000, 51090, 4.5, https://www.google.com//..."
8,Artis,Monument of Gebouw,<WikipediaPage 'Artis'>,https://nl.wikipedia.org/wiki/Artis,169659.0,0.339592,1406932.0,1.499159e-01,771,0.513333,...,7069.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36611111, 4.91583333)",POINT (4.91583333 52.36611111),Centrum,Weesperbuurt/Plantage,Plantage,0,12,"(5480000, 23501, 4.5, https://www.google.com//..."
9,Basiliek van de Heilige Nicolaas,Monument of Gebouw,<WikipediaPage 'Basiliek van de Heilige Nicola...,https://nl.wikipedia.org/wiki/Basiliek_van_de_...,48705.0,0.097456,14751.0,1.571796e-03,170,0.112667,...,493.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37652778, 4.90111111)",POINT (4.90111111 52.37

In [66]:
d2["google"] = d2["name"].apply(lambda a: scrape_google(a))

<ipython-input-66-e97070ac1039>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d2["google"] = d2["name"].apply(lambda a: scrape_google(a))


In [67]:
d2

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,TA_reviews,TA_url,wiki_coordinates,WKT_LNG_LAT,District,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google
20,De Papegaai,Monument of Gebouw,<WikipediaPage 'De Papegaai'>,https://nl.wikipedia.org/wiki/De_Papegaai,18235.0,0.036458,12778.0,0.001362,52,0.034000,...,62.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37111111, 4.89166667)",POINT (4.89166667 52.37111111),Centrum,Burgwallen-Nieuwe Zijde,Begijnhofbuurt,0,61,"(262000, nan, 4.6, https://www.google.com//map..."
21,De Wallen,Museum,<WikipediaPage 'De Wallen'>,https://nl.wikipedia.org/wiki/De_Wallen,241874.0,0.484158,1129722.0,0.120378,346,0.230000,...,31569.0,https://www.tripadvisor.nl/ShowUserReviews-g18...,"(52.37416667, 4.89944444)",POINT (4.89944444 52.37416667),Centrum,Burgwallen-Oude Zijde,Burgwallen Oost,0,111,"(4450000, nan, nan, https://www.google.com//ma..."
22,Eye Filmmuseum,Museum,<WikipediaPage 'Eye Filmmuseum'>,https://nl.wikipedia.org/wiki/Eye_Filmmuseum,1025.0,0.002006,117929.0,0.012566,218,0.144667,...,1234.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.38441111, 4.90059444)",POINT (4.90059444 52.38441111),Noord,Noordelijke IJ-oevers West,Overhoeks,0,0,"(204000, 7787, 4.4, https://www.google.com//ma..."
23,Engelse Hervormde Kerk,Monument of Gebouw,<WikipediaPage 'Engelse Hervormde Kerk (Amster...,https://nl.wikipedia.org/wiki/Engelse_Hervormd...,10790.0,0.021554,0.0,0.000000,65,0.042667,...,79.0,https://www.tripadvisor.nl/ShowUserReviews-g18...,"(52.36916667, 4.89)",POINT (4.89 52.36916667),Centrum,Burgwallen-Nieuwe Zijde,Begijnhofbuurt,5,110,"(288000, nan, 4.5, https://www.google.com//map..."
24,He Hua Tempel,Monument of Gebouw,<WikipediaPage 'He Hua Tempel'>,https://nl.wikipedia.org/wiki/He_Hua_Tempel,19229.0,0.038448,0.0,0.000000,73,0.048000,...,96.0,https://www.tripadvisor.nl/ShowUserReviews-g18...,"(52.37361111, 4.9)",POINT (4.9 52.37361111),Centrum,Burgwallen-Oude Zijde,Burgwallen Oost,0,116,"(689000, nan, 4.3, https://www.google.com//map..."
25,Hermitage Amsterdam,Museum,<WikipediaPage 'Hermitage Amsterdam'>,https://nl.wikipedia.org/wiki/Hermitage_Amsterdam,101045.0,0.202235,121837.0,0.012982,305,0.202667,...,2373.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.365, 4.9025)",POINT (4.9025 52.365),Centrum,Weesperbuurt/Plantage,Weesperbuurt,14,0,"(4710000, 6036, 4.5, https://www.google.com//m..."
26,Homomonument,Herdenkings Monument,<WikipediaPage 'Homomonument (Amsterdam)'>,https://nl.wikipedia.org/wiki/Homomonument_(Am...,26124.0,0.052251,96841.0,0.010319,208,0.138000,...,238.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37447222, 4.88461111)",POINT (4.88461111 52.37447222),Centrum,Grachtengordel-West,Leliegracht e.o.,44,0,"(109000, 1068, 4.3, https://www.google.com//ma..."
27,Kalverpassage,Monument of Gebouw,<WikipediaPage 'Kalverpassage'>,https://nl.wikipedia.org/wiki/Kalverpassage,2307.0,0.004572,0.0,0.000000,26,0.016667,...,50.0,https://www.tripadvisor.com/Attraction_Review-...,"(52.36743056, 4.892125)",POINT (4.892125 52.36743056),Centrum,Burgwallen-Nieuwe Zijde,Kalverdriehoek,4,47,"(10100, 2341, 4.0, https://www.google.com//map..."
28,Kleinste huis van Amsterdam,Monument of Gebouw,<WikipediaPage 'Kleinste huis van Amsterdam'>,https://nl.wikipedia.org/wiki/Kleinste_huis_va...,16274.0,0.032533,0.0,0.000000,28,0.018000,...,266.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37111111, 4.89777778)",POINT (4.89777778 52.37111111),Centrum,Burgwallen-Oude Zijde,BG-terrein e.o.,0,95,"(3280000, nan, 4.5, https://www.google.com//ma..."
29,Koninklijk Concertgebouw,Monument of Gebouw,<WikipediaPage 'Koninklijk Concertgebouw'>,https://nl.wikipedia.org/wiki/Koninklijk_Conce...,79353.0,0.158810,224763.0,0.023950,333,0.221333,...,3058.0,https://www.tripadvisor.nl/ShowUserReviews-g18...,"(52.35623889, 4.87908889)",POINT (4.87908889 52.35623889),Zuid,Museumkwar

In [73]:
d3["google"] = d3["name"].apply(lambda a: scrape_google(a))

<ipython-input-73-d6f18b76efac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d3["google"] = d3["name"].apply(lambda a: scrape_google(a))


In [74]:
d3

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,TA_reviews,TA_url,wiki_coordinates,WKT_LNG_LAT,District,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google
40,Museumplein,Monument of Gebouw,<WikipediaPage 'Museumplein'>,https://nl.wikipedia.org/wiki/Museumplein,54578.0,0.109213,118474.0,0.012624,224,0.148667,...,4704.0,https://www.tripadvisor.nl/ShowUserReviews-g18...,"(52.35761111, 4.88161944)",POINT (4.88161944 52.35761111),Zuid,Museumkwartier,Museumplein,0,1,"(1330000, nan, nan, https://www.google.com//ma..."
41,Muziekgebouw aan 't IJ,Monument of Gebouw,<WikipediaPage 'Muziekgebouw aan 't IJ'>,https://nl.wikipedia.org/wiki/Muziekgebouw_aan...,37433.0,0.074891,36887.0,0.003931,94,0.062000,...,59.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37833333, 4.91305556)",POINT (4.91305556 52.37833333),Oost,Oostelijk Havengebied,Oostelijke Handelskade,0,0,"(130000, 1176, 4.5, https://www.google.com//ma..."
42,NEMO Science Museum,Museum,<WikipediaPage 'NEMO Science Museum'>,https://nl.wikipedia.org/wiki/NEMO_Science_Museum,34425.0,0.068869,134487.0,0.014330,212,0.140667,...,4354.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37410556, 4.91239167)",POINT (4.91239167 52.37410556),Centrum,Nieuwmarkt/Lastage,Rapenburg,0,0,"(4290000, 22995, 4.5, https://www.google.com//..."
43,Nieuwe Kerk,Monument of Gebouw,<WikipediaPage 'Nieuwe Kerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Nieuwe_Kerk_(Ams...,97940.0,0.196019,29847.0,0.003180,266,0.176667,...,746.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37388889, 4.89194444)",POINT (4.89194444 52.37388889),Centrum,Burgwallen-Nieuwe Zijde,Nieuwe Kerk e.o.,0,56,"(8370000, 2413, 4.2, https://www.google.com//m..."
44,Noorderkerk,Monument of Gebouw,<WikipediaPage 'Noorderkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Noorderkerk_(Ams...,20121.0,0.040234,17004.0,0.001812,84,0.055333,...,51.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37944444, 4.88638889)",POINT (4.88638889 52.37944444),Centrum,Jordaan,Anjeliersbuurt Noord,55,7,"(169000, nan, 4.5, https://www.google.com//map..."
45,Noordermarkt,Markt,<WikipediaPage 'Noordermarkt'>,https://nl.wikipedia.org/wiki/Noordermarkt,18146.0,0.036280,25029.0,0.002667,61,0.040000,...,312.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37916667, 4.88611111)",POINT (4.88611111 52.37916667),Centrum,Jordaan,Anjeliersbuurt Noord,48,6,"(234000, nan, nan, https://www.google.com//map..."
46,Olympisch Stadion,Monument of Gebouw,<WikipediaPage 'Olympisch Stadion (Amsterdam)'>,https://nl.wikipedia.org/wiki/Olympisch_Stadio...,119368.0,0.238915,215010.0,0.022910,435,0.289333,...,129.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.34341944, 4.85419167)",POINT (4.85419167 52.34341944),Zuid,Stadionbuurt,Olympisch Stadion e.o.,0,4,"(981000, nan, 4.4, nan, nan)"
47,Oost-Indisch Huis,Monument of Gebouw,<WikipediaPage 'Oost-Indisch Huis (Amsterdam)'>,https://nl.wikipedia.org/wiki/Oost-Indisch_Hui...,37870.0,0.075765,60361.0,0.006432,86,0.056667,...,26.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37083333, 4.89805556)",POINT (4.89805556 52.37083333),Centrum,Burgwallen-Oude Zijde,BG-terrein e.o.,0,85,"(490000, nan, nan, https://www.google.com//map..."
48,Oosterkerk,Monument of Gebouw,<WikipediaPage 'Oosterkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Oosterkerk_(Amst...,13565.0,0.027110,7367.0,0.000785,92,0.060667,...,6.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36972222, 4.91916667)",POINT (4.91916667 52.36972222),Centrum,Oostelijke Eilanden/Kadijken,Wittenburg,0,12,"(87800, nan, 4.2, https://www.google.com//maps..."
49,Oude Kerk,Monument of Gebouw,<WikipediaPage 'Oude Kerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Oude_Kerk_(Amste...,96923.0,0.193983,138585.0,0.014767,253,0.168000,...,1101.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37416667, 4.89805556)",POINT (4.8980555

In [79]:
d4["google"] = d4["name"].apply(lambda a: scrape_google(a))

<ipython-input-79-dfcfd842d84e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d4["google"] = d4["name"].apply(lambda a: scrape_google(a))


In [80]:
d4

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,TA_reviews,TA_url,wiki_coordinates,WKT_LNG_LAT,District,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google
60,Station Amsterdam Centraal,Monument of Gebouw,<WikipediaPage 'Station Amsterdam Centraal'>,https://nl.wikipedia.org/wiki/Station_Amsterda...,182929.0,0.366157,183210.0,0.019522,999,0.665333,...,17480.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37888889, 4.90055556)",POINT (4.90055556 52.37888889),Centrum,Burgwallen-Nieuwe Zijde,Stationsplein e.o.,0,2,"(11100000, 4652, 4.5, https://www.google.com//..."
61,Stedelijk Museum Amsterdam,Museum,<WikipediaPage 'Stedelijk Museum Amsterdam'>,https://nl.wikipedia.org/wiki/Stedelijk_Museum...,90054.0,0.180232,190598.0,0.020309,481,0.320000,...,2949.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35777778, 4.87972222)",POINT (4.87972222 52.35777778),Zuid,Museumkwartier,Museumplein,0,2,"(3730000, 10455, 4.4, https://www.google.com//..."
62,Stopera,Monument of Gebouw,<WikipediaPage 'Stopera'>,https://nl.wikipedia.org/wiki/Stopera,72530.0,0.145151,0.0,0.000000,166,0.110000,...,1366.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36726709, 4.90153313)",POINT (4.90153313 52.36726709),Centrum,Nieuwmarkt/Lastage,Waterloopleinbuurt,0,1,"(263000, nan, 4.6, https://www.google.com//map..."
63,Theater Amsterdam,Monument of Gebouw,<WikipediaPage 'Theater Amsterdam'>,https://nl.wikipedia.org/wiki/Theater_Amsterdam,9363.0,0.018698,224693.0,0.023942,7,0.004000,...,149.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.39708056, 4.87789444)",POINT (4.87789444 52.39708056),Westpoort,Westelijk Havengebied,Coenhaven/Mercuriushaven,0,0,"(82500000, nan, 4.4, nan, nan)"
64,Theater Bellevue,Concertzaal of Theater,<WikipediaPage 'Theater Bellevue'>,https://nl.wikipedia.org/wiki/Theater_Bellevue,11797.0,0.023570,0.0,0.000000,64,0.042000,...,15.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36415833, 4.88027222)",POINT (4.88027222 52.36415833),Centrum,De Weteringschans,Leidsebuurt Zuidwest,0,11,"(5530000, nan, 4.4, https://www.google.com//ma..."
65,Tropenmuseum,Museum,<WikipediaPage 'Tropenmuseum'>,https://nl.wikipedia.org/wiki/Tropenmuseum,70790.0,0.141667,122105.0,0.013011,310,0.206000,...,1134.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.3625, 4.92111111)",POINT (4.92111111 52.3625),Oost,Oosterparkbuurt,Oosterpark,0,0,"(847000, 4027, 4.4, https://www.google.com//ma..."
66,Van Gogh Museum,Museum,<WikipediaPage 'Van Gogh Museum'>,https://nl.wikipedia.org/wiki/Van_Gogh_Museum,110539.0,0.221240,699953.0,0.074584,382,0.254000,...,63930.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35846111, 4.88127778)",POINT (4.88127778 52.35846111),Zuid,Museumkwartier,Museumplein,0,2,"(7310000, 64841, 4.6, https://www.google.com//..."
67,Verzetsmuseum Amsterdam,Museum,<WikipediaPage 'Verzetsmuseum Amsterdam'>,https://nl.wikipedia.org/wiki/Verzetsmuseum_Am...,17624.0,0.035235,45735.0,0.004873,43,0.028000,...,3416.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.3677, 4.91273611)",POINT (4.91273611 52.3677),Centrum,Weesperbuurt/Plantage,Plantage,0,14,"(160000, 1651, 4.4, https://www.google.com//ma..."
68,Westerkerk,Monument of Gebouw,<WikipediaPage 'Westerkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Westerkerk_(Amst...,60655.0,0.121378,132978.0,0.014169,231,0.153333,...,707.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.374525, 4.88399722)",POINT (4.88399722 52.374525),Centrum,Grachtengordel-West,Leliegracht e.o.,48,0,"(748000, 1030, 4.4, https://www.google.com//ma..."
69,Zeedijk,Monument of Gebouw,<WikipediaPage 'Zeedijk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Zeedijk_(Amsterdam),42813.0,0.085661,29933.0,0.003190,152,0.100667,...,236.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37406647, 4.90041667)",POINT (4.90041667 52.37406647),Centrum,Burgwallen-Oude Zijde,Burgwa

In [84]:
d5["google"] = d5["name"].apply(lambda a: scrape_google(a))

<ipython-input-84-1ab8d7db65a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d5["google"] = d5["name"].apply(lambda a: scrape_google(a))


In [85]:
d5

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,TA_reviews,TA_url,wiki_coordinates,WKT_LNG_LAT,District,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google
80,Zuiderbad,Monument of Gebouw,<WikipediaPage 'Zuiderbad'>,https://nl.wikipedia.org/wiki/Zuiderbad,26898.0,0.053801,0.0,0.000000,64,0.042000,...,25.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35833333, 4.88583333)",POINT (4.88583333 52.35833333),Zuid,Museumkwartier,Johannnes Vermeerbuurt,0,6,"(28600, nan, 4.3, https://www.google.com//maps..."
81,Blauwe Theehuis,Monument of Gebouw,<WikipediaPage 'Blauwe Theehuis'>,https://nl.wikipedia.org/wiki/Blauwe_Theehuis,12997.0,0.025972,12708.0,0.001354,34,0.022000,...,194.0,https://www.tripadvisor.nl/Restaurant_Review-g...,"(52.35881944, 4.87258333)",POINT (4.87258333 52.35881944),Zuid,Museumkwartier,Vondelpark Oost,0,2,"(73700, nan, 4.4, https://www.google.com//maps..."
82,Doelen Hotel,Monument of Gebouw,<WikipediaPage 'Doelen Hotel'>,https://nl.wikipedia.org/wiki/Doelen_Hotel,11904.0,0.023784,0.0,0.000000,55,0.036000,...,1020.0,https://www.tripadvisor.nl/Hotel_Review-g18859...,"(52.36777778, 4.89555556)",POINT (4.89555556 52.36777778),Centrum,Burgwallen-Oude Zijde,BG-terrein e.o.,0,28,"(800000, nan, 4.6, https://www.google.com//map..."
83,Vondelpark,Park of recreatiegebied,<WikipediaPage 'Vondelpark'>,https://nl.wikipedia.org/wiki/Vondelpark,108132.0,0.216422,268860.0,0.028648,361,0.240000,...,24728.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35699389, 4.86660003)",POINT (4.86660003 52.35699389),Zuid,Willemspark,Vondelpark West,0,1,"(2250000, 42138, 4.7, https://www.google.com//..."
84,Hollandsche Manege,Monument of Gebouw,<WikipediaPage 'Hollandsche Manege'>,https://nl.wikipedia.org/wiki/Hollandsche_Manege,15170.0,0.030323,12582.0,0.001341,77,0.050667,...,80.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36083333, 4.87222222)",POINT (4.87222222 52.36083333),West,Vondelbuurt,Vondelparkbuurt Midden,0,7,"(28900, nan, 3.7, https://www.google.com//maps..."
85,Huis met de Kabouters,Monument of Gebouw,<WikipediaPage 'Huis met de Kabouters'>,https://nl.wikipedia.org/wiki/Huis_met_de_Kabo...,7997.0,0.015963,5878.0,0.000626,31,0.020000,...,18.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35583333, 4.90444444)",POINT (4.90444444 52.35583333),Zuid,Oude Pijp,Hemonybuurt,0,6,"(80600, nan, 4.3, https://www.google.com//maps..."
86,Kerk van Durgerdam,Monument of Gebouw,<WikipediaPage 'Kerk van Durgerdam'>,https://nl.wikipedia.org/wiki/Kerk_van_Durgerdam,3779.0,0.007519,9384807.0,1.000000,8,0.004667,...,1.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37832222, 4.98813889)",POINT (4.98813889 52.37832222),Noord,Waterland,Durgerdam,0,2,"(44100, nan, 4.6, https://www.google.com//maps..."
87,Beurs van Berlage,Monument of Gebouw,<WikipediaPage 'Beurs van Berlage'>,https://nl.wikipedia.org/wiki/Beurs_van_Berlage,97893.0,0.195925,84574.0,0.009012,420,0.279333,...,19.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.374787, 4.89614725)",POINT (4.89614725 52.374787),Centrum,Burgwallen-Oude Zijde,Oude Kerk e.o.,0,54,"(270000, 3035, 4.4, https://www.google.com//ma..."
88,Hollandsche Schouwburg,Herdenkings Monument,<WikipediaPage 'Hollandsche Schouwburg'>,https://nl.wikipedia.org/wiki/Hollandsche_Scho...,56304.0,0.112668,19800.0,0.002110,180,0.119333,...,142.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.3664, 4.91115833)",POINT (4.91115833 52.3664),Centrum,Weesperbuurt/Plantage,Plantage,0,16,"(95200, nan, 4.4, https://www.google.com//maps..."
89,American Hotel,Monument of Gebouw,<WikipediaPage 'American Hotel'>,https://nl.wikipedia.org/wiki/American_Hotel,37073.0,0.074170,0.0,0.000000,105,0.069333,...,95.0,https://www.tripadvisor.nl/Hotel_Review-g18859...,"(52.36361111, 4.88111111)",POINT (4.88111111 52.36361111),Centrum,De Weteringschans,Leidsebuurt Zuidwest,0,6,"(59500000, nan, 4

In [86]:
d6["google"] = d6["name"].apply(lambda a: scrape_google(a))

<ipython-input-86-110485fca328>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d6["google"] = d6["name"].apply(lambda a: scrape_google(a))


In [87]:
d6

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,TA_reviews,TA_url,wiki_coordinates,WKT_LNG_LAT,District,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google
100,De 1200 Roe,Monument of Gebouw,<WikipediaPage 'De 1200 Roe'>,https://nl.wikipedia.org/wiki/De_1200_Roe,5298.0,0.010560,310903.0,3.312833e-02,41,0.026667,...,21.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.38416667, 4.815)",POINT (4.815 52.38416667),Nieuw-West,Slotermeer-Zuidwest,Buurt 4 Oost,0,1,"(693000, nan, 4.0, https://www.google.com//map..."
101,De Gooyer,Monument of Gebouw,<WikipediaPage 'De Gooyer (Amsterdam)'>,https://nl.wikipedia.org/wiki/De_Gooyer_(Amste...,19109.0,0.038208,41876.0,4.462106e-03,76,0.050000,...,308.0,https://www.tripadvisor.nl/ShowUserReviews-g18...,"(52.36666667, 4.92611111)",POINT (4.92611111 52.36666667),Centrum,Oostelijke Eilanden/Kadijken,Kazernebuurt,0,1,"(135000, nan, 4.4, https://www.google.com//map..."
102,Hilton Amsterdam Hotel,Monument of Gebouw,<WikipediaPage 'Hilton Amsterdam Hotel'>,https://nl.wikipedia.org/wiki/Hilton_Amsterdam...,53401.0,0.106857,69999.0,7.458758e-03,119,0.078667,...,3760.0,https://www.tripadvisor.nl/Hotel_Review-g18859...,"(52.35138889, 4.87194444)",POINT (4.87194444 52.35138889),Zuid,Apollobuurt,Hiltonbuurt,0,1,"(38100000, nan, 4.4, https://www.google.com//m..."
103,Aluminiumbrug,Monument of Gebouw,<WikipediaPage 'Aluminiumbrug'>,https://nl.wikipedia.org/wiki/Aluminiumbrug,2933.0,0.005825,0.0,0.000000e+00,13,0.008000,...,8.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36838333, 4.89629167)",POINT (4.89629167 52.36838333),Centrum,Burgwallen-Oude Zijde,BG-terrein e.o.,0,58,"(7090, nan, nan, https://www.google.com//maps/..."
104,Westergasfabriek,Monument of Gebouw,<WikipediaPage 'Westergasfabriek'>,https://nl.wikipedia.org/wiki/Westergasfabriek,72796.0,0.145683,24039.0,2.561480e-03,127,0.084000,...,159.0,https://www.tripadvisor.nl/ShowUserReviews-g18...,"(52.38666667, 4.87083333)",POINT (4.87083333 52.38666667),West,Spaarndammer- en Zeeheldenbuurt,Westergasfabriek,0,4,"(267000, nan, 4.3, https://www.google.com//map..."
105,Museumbrug,Monument of Gebouw,<WikipediaPage 'Museumbrug (Amsterdam)'>,https://nl.wikipedia.org/wiki/Museumbrug_(Amst...,2327.0,0.004612,0.0,0.000000e+00,18,0.011333,...,18.0,https://www.tripadvisor.com/AttractionProductR...,"(52.36078333, 4.88636389)",POINT (4.88636389 52.36078333),Centrum,De Weteringschans,Den Texbuurt,0,9,"(21400, nan, nan, https://www.google.com//maps..."
106,P.L. Kramerbrug,Monument of Gebouw,<WikipediaPage 'P.L. Kramerbrug'>,https://nl.wikipedia.org/wiki/P.L._Kramerbrug,4338.0,0.008638,0.0,0.000000e+00,29,0.018667,...,4.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.34986944, 4.90975278)",POINT (4.90975278 52.34986944),Zuid,Zuid Pijp,Diamantbuurt,0,6,"(3770, nan, nan, https://www.google.com//maps/..."
107,Amstel Hotel,Monument of Gebouw,<WikipediaPage 'Amstel Hotel'>,https://nl.wikipedia.org/wiki/Amstel_Hotel,67009.0,0.134098,34650.0,3.692138e-03,170,0.112667,...,1309.0,https://www.tripadvisor.nl/Hotel_Review-g18859...,"(52.35971944, 4.90591049)",POINT (4.90591049 52.35971944),Centrum,Weesperbuurt/Plantage,Sarphatistrook,0,3,"(3790000, nan, 4.6, https://www.google.com//ma..."
108,Nieuwer-Amstel,Monument of Gebouw,<WikipediaPage 'Nieuwer-Amstel'>,https://nl.wikipedia.org/wiki/Nieuwer-Amstel,32638.0,0.065292,349522.0,3.724339e-02,113,0.074667,...,11.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35333333, 4.90694444)",POINT (4.90694444 52.35333333),Zuid,Nieuwe Pijp,Willibrordusbuurt,0,4,"(4590000, nan, nan, https://www.google.com//ma..."
109,Rijksmuseumtuinen,Monument of Gebouw,<WikipediaPage 'Rijksmuseumtuinen'>,https://nl.wikipedia.org/wiki/Rijksmuseumtuinen,5216.0,0.010396,0.0,0.000000e+00,46,0.030000,...,22.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35983333, 4.885)",POINT (4.885 52.35983333),Zuid,Museumkwart

In [88]:
d7["google"] = d7["name"].apply(lambda a: scrape_google(a))

<ipython-input-88-175d065a5e45>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d7["google"] = d7["name"].apply(lambda a: scrape_google(a))


In [89]:
d7

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,TA_reviews,TA_url,wiki_coordinates,WKT_LNG_LAT,District,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google
120,Scheepvaarthuis,Monument of Gebouw,<WikipediaPage 'Scheepvaarthuis'>,https://nl.wikipedia.org/wiki/Scheepvaarthuis,43659.0,0.087354,41746.0,0.004448,130,0.086000,...,30.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37436111, 4.90410833)",POINT (4.90410833 52.37436111),Centrum,Nieuwmarkt/Lastage,Scheepvaarthuisbuurt,0,27,"(32900, nan, 4.3, https://www.google.com//maps..."
121,Grand Hotel Krasnapolsky,Monument of Gebouw,<WikipediaPage 'Grand Hotel Krasnapolsky'>,https://nl.wikipedia.org/wiki/Grand_Hotel_Kras...,25079.0,0.050159,24108.0,0.002569,88,0.058000,...,3972.0,https://www.tripadvisor.nl/Hotel_Review-g18859...,"(52.37277778, 4.89444444)",POINT (4.89444444 52.37277778),Centrum,Burgwallen-Oude Zijde,Oude Kerk e.o.,0,26,"(292000, nan, 4.5, https://www.google.com//map..."
122,Schreierstoren,Monument of Gebouw,<WikipediaPage 'Schreierstoren'>,https://nl.wikipedia.org/wiki/Schreierstoren,30137.0,0.060285,20362.0,0.002170,93,0.061333,...,142.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37638889, 4.9025)",POINT (4.9025 52.37638889),Centrum,Burgwallen-Oude Zijde,Kop Zeedijk,0,41,"(43700, nan, 4.4, https://www.google.com//maps..."
123,Trippenhuis,Monument of Gebouw,<WikipediaPage 'Trippenhuis'>,https://nl.wikipedia.org/wiki/Trippenhuis,44061.0,0.088159,29614.0,0.003156,127,0.084000,...,10.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37122222, 4.89939722)",POINT (4.89939722 52.37122222),Centrum,Nieuwmarkt/Lastage,Zuiderkerkbuurt,0,81,"(45600, nan, nan, https://www.google.com//maps..."
124,Auschwitzmonument,Herdenkings Monument,<WikipediaPage 'Auschwitzmonument (Amsterdam)'>,https://nl.wikipedia.org/wiki/Auschwitzmonumen...,39822.0,0.079673,0.0,0.000000,93,0.061333,...,37.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36777778, 4.90888889)",POINT (4.90888889 52.36777778),Centrum,Weesperbuurt/Plantage,Plantage,0,13,"(9100, nan, 4.4, https://www.google.com//maps/..."
125,De Dokwerker,Herdenkings Monument,<WikipediaPage 'De Dokwerker'>,https://nl.wikipedia.org/wiki/De_Dokwerker,52673.0,0.105399,75974.0,0.008095,111,0.073333,...,5.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36694444, 4.905)",POINT (4.905 52.36694444),Centrum,Nieuwmarkt/Lastage,Waterloopleinbuurt,6,17,"(34500, nan, 4.4, https://www.google.com//maps..."
126,Nationaal monument slavernijverleden,Herdenkings Monument,<WikipediaPage 'Nationaal monument slavernijve...,https://nl.wikipedia.org/wiki/Nationaal_monume...,23252.0,0.046502,0.0,0.000000,78,0.051333,...,7.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35861111, 4.91777778)",POINT (4.91777778 52.35861111),Oost,Oosterparkbuurt,Oosterpark,0,0,"(27200, nan, 4.4, https://www.google.com//maps..."
127,Museum Willet-Holthuysen,Museum,<WikipediaPage 'Museum Willet-Holthuysen'>,https://nl.wikipedia.org/wiki/Museum_Willet-Ho...,18868.0,0.037726,26002.0,0.002771,59,0.038667,...,768.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36555556, 4.89916667)",POINT (4.89916667 52.36555556),Centrum,Grachtengordel-Zuid,Rembrandtpleinbuurt,39,4,"(69200, nan, 4.5, https://www.google.com//maps..."
128,Heineken Experience,Museum,<WikipediaPage 'Heineken Experience'>,https://nl.wikipedia.org/wiki/Heineken_Experience,19809.0,0.039609,112455.0,0.011983,53,0.034667,...,25499.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35777778, 4.89111111)",POINT (4.89111111 52.35777778),Zuid,Oude Pijp,Gerard Doubuurt,0,6,"(6850000, 6575, 4.2, https://www.google.com//m..."
129,Huis Marseille,Museum,<WikipediaPage 'Huis Marseille'>,https://nl.wikipedia.org/wiki/Huis_Marseille,11996.0,0.023969,8051.0,0.000858,56,0.036667,...,176.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36756111, 4.88487778)",POINT (4.8

In [90]:
d8["google"] = d8["name"].apply(lambda a: scrape_google(a))

<ipython-input-90-0681def30e50>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d8["google"] = d8["name"].apply(lambda a: scrape_google(a))


In [91]:
d8

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,TA_reviews,TA_url,wiki_coordinates,WKT_LNG_LAT,District,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google
140,De Meervaart,Concertzaal of Theater,<WikipediaPage 'De Meervaart'>,https://nl.wikipedia.org/wiki/De_Meervaart,11950.0,0.023876,81444.0,0.008678,52,0.034000,...,3.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35888889, 4.80722222)",POINT (4.80722222 52.35888889),Nieuw-West,Osdorp-Oost,Osdorpplein e.o.,0,0,"(101000, 1725, 4.4, https://www.google.com//ma..."
141,Theater Fabriek Amsterdam,Concertzaal of Theater,<WikipediaPage 'Theater Fabriek Amsterdam'>,https://nl.wikipedia.org/wiki/Theater_Fabriek_...,3825.0,0.007611,0.0,0.000000,25,0.016000,...,149.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37266944, 4.92790833)",POINT (4.92790833 52.37266944),Centrum,Oostelijke Eilanden/Kadijken,Oostenburg,0,1,"(495000, nan, 3.5, https://www.google.com//map..."
142,Ostadetheater,Concertzaal of Theater,<WikipediaPage 'Ostadetheater'>,https://nl.wikipedia.org/wiki/Ostadetheater,2940.0,0.005840,0.0,0.000000,18,0.011333,...,1.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35333333, 4.89888889)",POINT (4.89888889 52.35333333),Zuid,Nieuwe Pijp,Van der Helstpleinbuurt,0,1,"(5780, nan, 4.3, https://www.google.com//maps/..."
143,Tropentheater,Concertzaal of Theater,<WikipediaPage 'Tropentheater'>,https://nl.wikipedia.org/wiki/Tropentheater,3455.0,0.006870,226642.0,0.024150,31,0.020000,...,2.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36259444, 4.92275278)",POINT (4.92275278 52.36259444),Oost,Oosterparkbuurt,Oosterpark,0,1,"(9720, nan, nan, https://www.google.com//maps/..."
144,The Movies,Bioscoop,<WikipediaPage 'The Movies (Amsterdam)'>,https://nl.wikipedia.org/wiki/The_Movies_(Amst...,12937.0,0.025852,9384807.0,1.000000,51,0.033333,...,29.0,https://www.tripadvisor.nl/Restaurant_Review-g...,"(52.38403333, 4.88468889)",POINT (4.88468889 52.38403333),Centrum,Haarlemmerbuurt,Haarlemmerbuurt West,0,28,"(72700000, nan, 4.6, https://www.google.com//m..."
145,Filmtheater Kriterion,Bioscoop,<WikipediaPage 'Filmtheater Kriterion'>,https://nl.wikipedia.org/wiki/Filmtheater_Krit...,9148.0,0.018267,117929.0,0.012566,107,0.070667,...,35.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.3625, 4.91055556)",POINT (4.91055556 52.3625),Centrum,Weesperbuurt/Plantage,Sarphatistrook,2,4,"(12800, 1343, 4.5, https://www.google.com//map..."
146,Filmtheater de Uitkijk,Bioscoop,<WikipediaPage 'Filmtheater de Uitkijk'>,https://nl.wikipedia.org/wiki/Filmtheater_de_U...,9657.0,0.019286,53163.0,0.005665,53,0.034667,...,11.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36479, 4.88434)",POINT (4.88434 52.36479),Centrum,De Weteringschans,Leidsebuurt Noordoost,47,12,"(11100, nan, 4.5, https://www.google.com//maps..."
147,Rialto,Bioscoop,<WikipediaPage 'Rialto (filmtheater)'>,https://nl.wikipedia.org/wiki/Rialto_(filmthea...,9664.0,0.019300,25782.0,0.002747,29,0.018667,...,16.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35305556, 4.89388889)",POINT (4.89388889 52.35305556),Zuid,Nieuwe Pijp,Lizzy Ansinghbuurt,0,1,"(2210000, nan, 4.4, https://www.google.com//ma..."
148,Amsterdamse Bos,Park of recreatiegebied,<WikipediaPage 'Amsterdamse Bos'>,https://nl.wikipedia.org/wiki/Amsterdamse_Bos,67699.0,0.135480,58196.0,0.006201,253,0.168000,...,774.0,https://www.tripadvisor.com/Attraction_Review-...,"(52.31083333, 4.8325)",POINT (4.8325 52.31083333),NaN,NaN,NaN,0,0,"(1230000, nan, 4.6, https://www.google.com//ma..."
149,Baanakkerspark,Park of recreatiegebied,<WikipediaPage 'Baanakkerspark'>,https://nl.wikipedia.org/wiki/Baanakkerspark,4009.0,0.007980,63068.0,0.006720,33,0.021333,...,8.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.39733611, 4.94732222)",POINT (4.94732222 52.39733611),Noord,Waterlandpleinbuurt,Werengouw Noord,0,0,"(12600, nan, nan, h

In [92]:
d9["google"] = d9["name"].apply(lambda a: scrape_google(a))

<ipython-input-92-3e2818e3fb6d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d9["google"] = d9["name"].apply(lambda a: scrape_google(a))


In [93]:
d9

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,TA_reviews,TA_url,wiki_coordinates,WKT_LNG_LAT,District,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google
160,Sarphatipark,Park of recreatiegebied,<WikipediaPage 'Sarphatipark'>,https://nl.wikipedia.org/wiki/Sarphatipark,25020.0,0.050041,23766.0,0.002532,70,0.046000,...,135.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35444444, 4.89638889)",POINT (4.89638889 52.35444444),Zuid,Oude Pijp,Sarphatiparkbuurt,0,14,"(134000, nan, nan, https://www.google.com//map..."
161,Vliegenbos,Park of recreatiegebied,<WikipediaPage 'Vliegenbos'>,https://nl.wikipedia.org/wiki/Vliegenbos,16650.0,0.033285,63068.0,0.006720,81,0.053333,...,968.0,https://www.tripadvisor.nl/Hotel_Review-g18859...,"(52.38944444, 4.93166667)",POINT (4.93166667 52.38944444),Noord,IJplein/Vogelbuurt,Vliegenbos,0,0,"(25900, nan, 4.4, nan, nan)"
162,Volewijkspark,Park of recreatiegebied,<WikipediaPage 'Volewijkspark'>,https://nl.wikipedia.org/wiki/Volewijkspark,3251.0,0.006462,63068.0,0.006720,14,0.008667,...,8.0,https://www.tripadvisor.nl/ShowUserReviews-g18...,"(52.39139444, 4.92337778)",POINT (4.92337778 52.39139444),Noord,IJplein/Vogelbuurt,Vogelbuurt Noord,0,0,"(1900, 2320, 4.4, https://www.google.com//maps..."
163,De Oeverlanden,Park of recreatiegebied,<WikipediaPage 'De Oeverlanden'>,https://nl.wikipedia.org/wiki/De_Oeverlanden,15578.0,0.031139,0.0,0.000000,40,0.026000,...,5.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.33611389, 4.81891111)",POINT (4.81891111 52.33611389),Nieuw-West,Sloter-/Riekerpolder,Nieuwe Meer,0,0,"(55800, 1210, 4.4, https://www.google.com//map..."
164,Sloterplas,Park of recreatiegebied,<WikipediaPage 'Sloterplas'>,https://nl.wikipedia.org/wiki/Sloterplas,26669.0,0.053342,362439.0,0.038620,103,0.068000,...,10.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36472222, 4.81694444)",POINT (4.81694444 52.36472222),Nieuw-West,Slotermeer-Zuidwest,Sloterpark,0,0,"(129000, nan, 4.5, nan, nan)"
165,Wertheimpark,Park of recreatiegebied,<WikipediaPage 'Wertheimpark'>,https://nl.wikipedia.org/wiki/Wertheimpark,19979.0,0.039950,5191.0,0.000553,55,0.036000,...,25.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36805556, 4.90916667)",POINT (4.90916667 52.36805556),Centrum,Weesperbuurt/Plantage,Plantage,0,14,"(27200, nan, 4.5, https://www.google.com//maps..."
166,Rembrandtpark,Park of recreatiegebied,<WikipediaPage 'Rembrandtpark'>,https://nl.wikipedia.org/wiki/Rembrandtpark,16900.0,0.033786,0.0,0.000000,42,0.027333,...,85.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36277778, 4.84555556)",POINT (4.84555556 52.36277778),Nieuw-West,Overtoomse Veld,Rembrandtpark Zuid,0,0,"(231000, 3882, 4.5, https://www.google.com//ma..."
167,De Hoge Dijk,Park of recreatiegebied,<WikipediaPage 'De Hoge Dijk'>,https://nl.wikipedia.org/wiki/De_Hoge_Dijk,5834.0,0.011633,0.0,0.000000,23,0.014667,...,19.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.28479167, 4.96781111)",POINT (4.96781111 52.28479167),Zuidoost,Amstel III/Bullewijk,Hoge Dijk,0,0,"(1870000, nan, 4.3, https://www.google.com//ma..."
168,Siegerpark,Park of recreatiegebied,<WikipediaPage 'Siegerpark'>,https://nl.wikipedia.org/wiki/Siegerpark,7495.0,0.014958,150247.0,0.016010,26,0.016667,...,6.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.34068889, 4.81939444)",POINT (4.81939444 52.34068889),Nieuw-West,Sloter-/Riekerpolder,Sloterweg e.o.,0,0,"(6120, nan, 4.4, https://www.google.com//maps/..."
169,Noorderpark,Park of recreatiegebied,<WikipediaPage 'Noorderpark (Amsterdam)'>,https://nl.wikipedia.org/wiki/Noorderpark_(Ams...,8852.0,0.017675,0.0,0.000000,37,0.024000,...,8.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.39472222, 4.91944444)",POINT (4.91944444 52.39472222),Noord,Volewijck,Bloemenbuurt Zuid,0,1,"(84400, 2320, 4.4, https://www.google.com//map..."


In [97]:
final = pd.concat([d1,d2,d3,d4,d5,d6,d7,d8,d9])
final

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,TA_reviews,TA_url,wiki_coordinates,WKT_LNG_LAT,District,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google
0,AFAS Live,Monument of Gebouw,<WikipediaPage 'AFAS Live'>,https://nl.wikipedia.org/wiki/AFAS_Live,124539.0,0.249267,105429.0,1.123401e-02,45,0.029333,...,260.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.31222222, 4.94416667)",POINT (4.94416667 52.31222222),Zuidoost,Amstel III/Bullewijk,Hoofdcentrum Zuidoost,0,0,"(2070000, 9285, 4.3, https://www.google.com//m..."
1,Albert Cuypmarkt,Markt,<WikipediaPage 'Albert Cuypmarkt'>,https://nl.wikipedia.org/wiki/Albert_Cuypmarkt,58069.0,0.116201,110365.0,1.175996e-02,70,0.046000,...,2428.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.35597222, 4.89538889)",POINT (4.89538889 52.35597222),Zuid,Oude Pijp,Sarphatiparkbuurt,0,3,"(109000, 25906, 4.3, https://www.google.com//m..."
2,Allard Pierson Museum,Museum,<WikipediaPage 'Allard Pierson Museum'>,https://nl.wikipedia.org/wiki/Allard_Pierson_M...,33905.0,0.067828,27502.0,2.930481e-03,109,0.072000,...,172.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36861111, 4.89333333)",POINT (4.89333333 52.36861111),Centrum,Burgwallen-Oude Zijde,BG-terrein e.o.,0,44,"(213000, nan, nan, https://www.google.com//map..."
3,Amstelkerk,Monument of Gebouw,<WikipediaPage 'Amstelkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amstelkerk_(Amst...,13384.0,0.026747,2.0,2.131104e-07,52,0.034000,...,6.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36222222, 4.89638889)",POINT (4.89638889 52.36222222),Centrum,Grachtengordel-Zuid,Amstelveldbuurt,56,0,"(71300, nan, 4.3, https://www.google.com//maps..."
4,Amstelveld,Markt,<WikipediaPage 'Amstelveld'>,https://nl.wikipedia.org/wiki/Amstelveld,20987.0,0.041968,53163.0,5.664794e-03,47,0.030667,...,12.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.36222222, 4.8975)",POINT (4.8975 52.36222222),Centrum,Grachtengordel-Zuid,Amstelveldbuurt,43,0,"(54800, nan, nan, https://www.google.com//maps..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,Siegerpark,Park of recreatiegebied,<WikipediaPage 'Siegerpark'>,https://nl.wikipedia.org/wiki/Siegerpark,7495.0,0.014958,150247.0,1.600960e-02,26,0.016667,...,6.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.34068889, 4.81939444)",POINT (4.81939444 52.34068889),Nieuw-West,Sloter-/Riekerpolder,Sloterweg e.o.,0,0,"(6120, nan, 4.4, https://www.google.com//maps/..."
169,Noorderpark,Park of recreatiegebied,<WikipediaPage 'Noorderpark (Amsterdam)'>,https://nl.wikipedia.org/wiki/Noorderpark_(Ams...,8852.0,0.017675,0.0,0.000000e+00,37,0.024000,...,8.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.39472222, 4.91944444)",POINT (4.91944444 52.39472222),Noord,Volewijck,Bloemenbuurt Zuid,0,1,"(84400, 2320, 4.4, https://www.google.com//map..."
170,Nieuwmarkt,Markt,<WikipediaPage 'Nieuwmarkt (Amsterdam)'>,https://nl.wikipedia.org/wiki/Nieuwmarkt_(Amst...,43629.0,0.087294,58019.0,6.182226e-03,137,0.090667,...,333.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.37257306, 4.90050316)",POINT (4.90050316 52.37257306),Centrum,Nieuwmarkt/Lastage,Nieuwmarkt,0,73,"(1650000, nan, nan, https://www.google.com//ma..."
171,Amsterdamse Poort,Markt,<WikipediaPage 'Amsterdamse Poort (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amsterdamse_Poor...,17723.0,0.035433,0.0,0.000000e+00,54,0.035333,...,10.0,https://www.tripadvisor.nl/Attraction_Review-g...,"(52.31388889, 4.9525)",POINT (4.9525 52.31388889),Zuidoost,"Bijlmer Centrum (D,F,H)",Amsterdamse Poort,0,0,"(328000, 6956, 4.1, https://www.google.com//ma..."


In [81]:
d12 = pd.concat([d1,d2])
d12["g_results"] = d12["google"].apply(lambda a: a[0])
d12["g_reviews"] = d12["google"].apply(lambda a: a[1])
d12["g_rating"] = d12["google"].apply(lambda a: a[2])
d12["g_link"] = d12["google"].apply(lambda a: a[3])
d12["g_coords"] = d12["google"].apply(lambda a: a[4])
d12

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google,g_results,g_reviews,g_rating,g_link,g_coords
0,AFAS Live,Monument of Gebouw,<WikipediaPage 'AFAS Live'>,https://nl.wikipedia.org/wiki/AFAS_Live,124539.0,0.249267,105429.0,1.123401e-02,45,0.029333,...,Amstel III/Bullewijk,Hoofdcentrum Zuidoost,0,0,"(2070000, 9285, 4.3, https://www.google.com//m...",2070000,9285.0,4.3,https://www.google.com//maps/place/AFAS+Live/@...,"(52.3119125, 4.9445428)"
1,Albert Cuypmarkt,Markt,<WikipediaPage 'Albert Cuypmarkt'>,https://nl.wikipedia.org/wiki/Albert_Cuypmarkt,58069.0,0.116201,110365.0,1.175996e-02,70,0.046000,...,Oude Pijp,Sarphatiparkbuurt,0,3,"(109000, 25906, 4.3, https://www.google.com//m...",109000,25906.0,4.3,https://www.google.com//maps/place/Albert+Cuyp...,"(52.3559673, 4.8953871)"
2,Allard Pierson Museum,Museum,<WikipediaPage 'Allard Pierson Museum'>,https://nl.wikipedia.org/wiki/Allard_Pierson_M...,33905.0,0.067828,27502.0,2.930481e-03,109,0.072000,...,Burgwallen-Oude Zijde,BG-terrein e.o.,0,44,"(213000, nan, nan, https://www.google.com//map...",213000,NaN,NaN,https://www.google.com//maps/place/Allard+Pier...,"(52.3687868, 4.8930188)"
3,Amstelkerk,Monument of Gebouw,<WikipediaPage 'Amstelkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amstelkerk_(Amst...,13384.0,0.026747,2.0,2.131104e-07,52,0.034000,...,Grachtengordel-Zuid,Amstelveldbuurt,56,0,"(71300, nan, 4.3, https://www.google.com//maps...",71300,NaN,4.3,https://www.google.com//maps/place/Amstelkerk+...,"(52.3623276, 4.8966565)"
4,Amstelveld,Markt,<WikipediaPage 'Amstelveld'>,https://nl.wikipedia.org/wiki/Amstelveld,20987.0,0.041968,53163.0,5.664794e-03,47,0.030667,...,Grachtengordel-Zuid,Amstelveldbuurt,43,0,"(54800, nan, nan, https://www.google.com//maps...",54800,NaN,NaN,"https://www.google.com//maps/place/Amstelveld,...",NaN
5,Johan Cruijff ArenA,Monument of Gebouw,<WikipediaPage 'Johan Cruijff ArenA'>,https://nl.wikipedia.org/wiki/Johan_Cruijff_ArenA,184052.0,0.368405,433884.0,4.623260e-02,1501,1.000000,...,Amstel III/Bullewijk,Hoofdcentrum Zuidoost,0,0,"(1630000, 28005, 4.4, https://www.google.com//...",1630000,28005.0,4.4,https://www.google.com//maps/place/Johan+Cruij...,"(52.3143713, 4.9419641)"
6,Amsterdam Museum,Museum,<WikipediaPage 'Amsterdam Museum'>,https://nl.wikipedia.org/wiki/Amsterdam_Museum,53772.0,0.107599,91050.0,9.701851e-03,277,0.184000,...,Burgwallen-Nieuwe Zijde,Begijnhofbuurt,0,96,"(97100000, 5328, 4.3, https://www.google.com//...",97100000,5328.0,4.3,https://www.google.com//maps/place/Amsterdam+M...,"(52.3699874, 4.8899741)"
7,Anne Frank Huis,Monument of Gebouw,<WikipediaPage 'Anne Frank Huis'>,https://nl.wikipedia.org/wiki/Anne_Frank_Huis,177070.0,0.354428,1121274.0,1.194776e-01,532,0.354000,...,Grachtengordel-West,Leliegracht e.o.,51,0,"(3090000, 51090, 4.5, https://www.google.com//...",3090000,51090.0,4.5,https://www.google.com//maps/place/Anne+Frank+...,"(52.3752182, 4.8839765)"
8,Artis,Monument of Gebouw,<WikipediaPage 'Artis'>,https://nl.wikipedia.org/wiki/Artis,169659.0,0.339592,1406932.0,1.499159e-01,771,0.513333,...,Weesperbuurt/Plantage,Plantage,0,12,"(5480000, 23501, 4.5, https://www.google.com//...",5480000,23501.0,4.5,https://www.google.com//maps/place/ARTIS/@52.3...,"(52.3660003, 4.9165321)"
9,Basiliek van de Heilige Nicolaas,Monument of Gebouw,<WikipediaPage 'Basiliek van de Heilige Nicola...,https://nl.wikipedia.org/wiki/Basiliek_van_de_...,48705.0,0.097456,14751.0,1.571796e-03,170,0.112667,...,Burgwallen-Oude Zijde,Kop Zeedijk,0,67,"(46500, 1929, 4.6, https://www.google.com//map...",46500,1929.0,4.6,https://www.google.com//maps/place/Basiliek+va...,"(52.3765536, 4.9008726)"


In [83]:
d12["g_norm_rating"] = (d12["g_rating"]-d12["g_rating"].min())/(d12["g_rating"].max()-d12["g_rating"].min())
d12


,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Neighborhood,UNESCO,Monuments,google,g_results,g_reviews,g_rating,g_link,g_coords,g_norm_rating
0,AFAS Live,Monument of Gebouw,<WikipediaPage 'AFAS Live'>,https://nl.wikipedia.org/wiki/AFAS_Live,124539.0,0.249267,105429.0,1.123401e-02,45,0.029333,...,Hoofdcentrum Zuidoost,0,0,"(2070000, 9285, 4.3, https://www.google.com//m...",2070000,9285.0,4.3,https://www.google.com//maps/place/AFAS+Live/@...,"(52.3119125, 4.9445428)",0.500
1,Albert Cuypmarkt,Markt,<WikipediaPage 'Albert Cuypmarkt'>,https://nl.wikipedia.org/wiki/Albert_Cuypmarkt,58069.0,0.116201,110365.0,1.175996e-02,70,0.046000,...,Sarphatiparkbuurt,0,3,"(109000, 25906, 4.3, https://www.google.com//m...",109000,25906.0,4.3,https://www.google.com//maps/place/Albert+Cuyp...,"(52.3559673, 4.8953871)",0.500
2,Allard Pierson Museum,Museum,<WikipediaPage 'Allard Pierson Museum'>,https://nl.wikipedia.org/wiki/Allard_Pierson_M...,33905.0,0.067828,27502.0,2.930481e-03,109,0.072000,...,BG-terrein e.o.,0,44,"(213000, nan, nan, https://www.google.com//map...",213000,NaN,NaN,https://www.google.com//maps/place/Allard+Pier...,"(52.3687868, 4.8930188)",NaN
3,Amstelkerk,Monument of Gebouw,<WikipediaPage 'Amstelkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amstelkerk_(Amst...,13384.0,0.026747,2.0,2.131104e-07,52,0.034000,...,Amstelveldbuurt,56,0,"(71300, nan, 4.3, https://www.google.com//maps...",71300,NaN,4.3,https://www.google.com//maps/place/Amstelkerk+...,"(52.3623276, 4.8966565)",0.500
4,Amstelveld,Markt,<WikipediaPage 'Amstelveld'>,https://nl.wikipedia.org/wiki/Amstelveld,20987.0,0.041968,53163.0,5.664794e-03,47,0.030667,...,Amstelveldbuurt,43,0,"(54800, nan, nan, https://www.google.com//maps...",54800,NaN,NaN,"https://www.google.com//maps/place/Amstelveld,...",NaN,NaN
5,Johan Cruijff ArenA,Monument of Gebouw,<WikipediaPage 'Johan Cruijff ArenA'>,https://nl.wikipedia.org/wiki/Johan_Cruijff_ArenA,184052.0,0.368405,433884.0,4.623260e-02,1501,1.000000,...,Hoofdcentrum Zuidoost,0,0,"(1630000, 28005, 4.4, https://www.google.com//...",1630000,28005.0,4.4,https://www.google.com//maps/place/Johan+Cruij...,"(52.3143713, 4.9419641)",0.625
6,Amsterdam Museum,Museum,<WikipediaPage 'Amsterdam Museum'>,https://nl.wikipedia.org/wiki/Amsterdam_Museum,53772.0,0.107599,91050.0,9.701851e-03,277,0.184000,...,Begijnhofbuurt,0,96,"(97100000, 5328, 4.3, https://www.google.com//...",97100000,5328.0,4.3,https://www.google.com//maps/place/Amsterdam+M...,"(52.3699874, 4.8899741)",0.500
7,Anne Frank Huis,Monument of Gebouw,<WikipediaPage 'Anne Frank Huis'>,https://nl.wikipedia.org/wiki/Anne_Frank_Huis,177070.0,0.354428,1121274.0,1.194776e-01,532,0.354000,...,Leliegracht e.o.,51,0,"(3090000, 51090, 4.5, https://www.google.com//...",3090000,51090.0,4.5,https://www.google.com//maps/place/Anne+Frank+...,"(52.3752182, 4.8839765)",0.750
8,Artis,Monument of Gebouw,<WikipediaPage 'Artis'>,https://nl.wikipedia.org/wiki/Artis,169659.0,0.339592,1406932.0,1.499159e-01,771,0.513333,...,Plantage,0,12,"(5480000, 23501, 4.5, https://www.google.com//...",5480000,23501.0,4.5,https://www.google.com//maps/place/ARTIS/@52.3...,"(52.3660003, 4.9165321)",0.750
9,Basiliek van de Heilige Nicolaas,Monument of Gebouw,<WikipediaPage 'Basiliek van de Heilige Nicola...,https://nl.wikipedia.org/wiki/Basiliek_van_de_...,48705.0,0.097456,14751.0,1.571796e-03,170,0.112667,...,Kop Zeedijk,0,67,"(46500, 1929, 4.6, https://www.google.com//map...",46500,1929.0,4.6,https://www.google.com//maps/place/Basiliek+va...,"(52.3765536, 4.9008726)",0.875


In [98]:
final["g_results"] = final["google"].apply(lambda a: a[0])
final["g_reviews"] = final["google"].apply(lambda a: a[1])
final["g_rating"] = final["google"].apply(lambda a: a[2])
final["g_link"] = final["google"].apply(lambda a: a[3])
final["g_coords"] = final["google"].apply(lambda a: a[4])
final

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google,g_results,g_reviews,g_rating,g_link,g_coords
0,AFAS Live,Monument of Gebouw,<WikipediaPage 'AFAS Live'>,https://nl.wikipedia.org/wiki/AFAS_Live,124539.0,0.249267,105429.0,1.123401e-02,45,0.029333,...,Amstel III/Bullewijk,Hoofdcentrum Zuidoost,0,0,"(2070000, 9285, 4.3, https://www.google.com//m...",2070000,9285.0,4.3,https://www.google.com//maps/place/AFAS+Live/@...,"(52.3119125, 4.9445428)"
1,Albert Cuypmarkt,Markt,<WikipediaPage 'Albert Cuypmarkt'>,https://nl.wikipedia.org/wiki/Albert_Cuypmarkt,58069.0,0.116201,110365.0,1.175996e-02,70,0.046000,...,Oude Pijp,Sarphatiparkbuurt,0,3,"(109000, 25906, 4.3, https://www.google.com//m...",109000,25906.0,4.3,https://www.google.com//maps/place/Albert+Cuyp...,"(52.3559673, 4.8953871)"
2,Allard Pierson Museum,Museum,<WikipediaPage 'Allard Pierson Museum'>,https://nl.wikipedia.org/wiki/Allard_Pierson_M...,33905.0,0.067828,27502.0,2.930481e-03,109,0.072000,...,Burgwallen-Oude Zijde,BG-terrein e.o.,0,44,"(213000, nan, nan, https://www.google.com//map...",213000,NaN,NaN,https://www.google.com//maps/place/Allard+Pier...,"(52.3687868, 4.8930188)"
3,Amstelkerk,Monument of Gebouw,<WikipediaPage 'Amstelkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amstelkerk_(Amst...,13384.0,0.026747,2.0,2.131104e-07,52,0.034000,...,Grachtengordel-Zuid,Amstelveldbuurt,56,0,"(71300, nan, 4.3, https://www.google.com//maps...",71300,NaN,4.3,https://www.google.com//maps/place/Amstelkerk+...,"(52.3623276, 4.8966565)"
4,Amstelveld,Markt,<WikipediaPage 'Amstelveld'>,https://nl.wikipedia.org/wiki/Amstelveld,20987.0,0.041968,53163.0,5.664794e-03,47,0.030667,...,Grachtengordel-Zuid,Amstelveldbuurt,43,0,"(54800, nan, nan, https://www.google.com//maps...",54800,NaN,NaN,"https://www.google.com//maps/place/Amstelveld,...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,Siegerpark,Park of recreatiegebied,<WikipediaPage 'Siegerpark'>,https://nl.wikipedia.org/wiki/Siegerpark,7495.0,0.014958,150247.0,1.600960e-02,26,0.016667,...,Sloter-/Riekerpolder,Sloterweg e.o.,0,0,"(6120, nan, 4.4, https://www.google.com//maps/...",6120,NaN,4.4,https://www.google.com//maps/place/Siegerpark/...,"(52.3407925, 4.8190865)"
169,Noorderpark,Park of recreatiegebied,<WikipediaPage 'Noorderpark (Amsterdam)'>,https://nl.wikipedia.org/wiki/Noorderpark_(Ams...,8852.0,0.017675,0.0,0.000000e+00,37,0.024000,...,Volewijck,Bloemenbuurt Zuid,0,1,"(84400, 2320, 4.4, https://www.google.com//map...",84400,2320.0,4.4,https://www.google.com//maps/place/Noorderpark...,"(52.3958556, 4.9200389)"
170,Nieuwmarkt,Markt,<WikipediaPage 'Nieuwmarkt (Amsterdam)'>,https://nl.wikipedia.org/wiki/Nieuwmarkt_(Amst...,43629.0,0.087294,58019.0,6.182226e-03,137,0.090667,...,Nieuwmarkt/Lastage,Nieuwmarkt,0,73,"(1650000, nan, nan, https://www.google.com//ma...",1650000,NaN,NaN,"https://www.google.com//maps/place/Nieuwmarkt,...",NaN
171,Amsterdamse Poort,Markt,<WikipediaPage 'Amsterdamse Poort (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amsterdamse_Poor...,17723.0,0.035433,0.0,0.000000e+00,54,0.035333,...,"Bijlmer Centrum (D,F,H)",Amsterdamse Poort,0,0,"(328000, 6956, 4.1, https://www.google.com//ma...",328000,6956.0,4.1,https://www.google.com//maps/place/Amsterdamse...,"(52.3140525, 4.9527714)"


In [99]:
final.isna().sum()/len(final)

name                         0.000000
label                        0.000000
wiki_page_obj                0.000000
wiki_url                     0.000000
wiki_nl_views                0.000000
wiki_norm_nl_views           0.000000
wiki_eng_views               0.000000
wiki_norm_eng_views          0.000000
wiki_nr_of_edits             0.000000
wiki_norm_nr_of_edits        0.000000
wiki_nr_of_languages         0.000000
wiki_norm_nr_of_languages    0.000000
TA_name                      0.000000
TA_rating                    0.000000
TA_reviews                   0.000000
TA_url                       0.000000
wiki_coordinates             0.000000
WKT_LNG_LAT                  0.000000
District                     0.005780
Neighborhoodcombination      0.005780
Neighborhood                 0.005780
UNESCO                       0.000000
Monuments                    0.000000
google                       0.000000
g_results                    0.000000
g_reviews                    0.653179
g_rating    

In [100]:
final.to_csv("fulldata_draft2.csv", index=False)

In [104]:
miss = final[final["g_reviews"].isna()]

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google,g_results,g_reviews,g_rating,g_link,g_coords
2,Allard Pierson Museum,Museum,<WikipediaPage 'Allard Pierson Museum'>,https://nl.wikipedia.org/wiki/Allard_Pierson_M...,33905.0,0.067828,27502.0,2.930481e-03,109,0.072000,...,Burgwallen-Oude Zijde,BG-terrein e.o.,0,44,"(213000, nan, nan, https://www.google.com//map...",213000,NaN,NaN,https://www.google.com//maps/place/Allard+Pier...,"(52.3687868, 4.8930188)"
3,Amstelkerk,Monument of Gebouw,<WikipediaPage 'Amstelkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amstelkerk_(Amst...,13384.0,0.026747,2.0,2.131104e-07,52,0.034000,...,Grachtengordel-Zuid,Amstelveldbuurt,56,0,"(71300, nan, 4.3, https://www.google.com//maps...",71300,NaN,4.3,https://www.google.com//maps/place/Amstelkerk+...,"(52.3623276, 4.8966565)"
4,Amstelveld,Markt,<WikipediaPage 'Amstelveld'>,https://nl.wikipedia.org/wiki/Amstelveld,20987.0,0.041968,53163.0,5.664794e-03,47,0.030667,...,Grachtengordel-Zuid,Amstelveldbuurt,43,0,"(54800, nan, nan, https://www.google.com//maps...",54800,NaN,NaN,"https://www.google.com//maps/place/Amstelveld,...",NaN
11,Bijbels Museum,Museum,<WikipediaPage 'Bijbels Museum'>,https://nl.wikipedia.org/wiki/Bijbels_Museum,11725.0,0.023426,17952.0,1.912879e-03,81,0.053333,...,Grachtengordel-West,Leidsegracht Noord,85,0,"(124000, nan, 4.2, https://www.google.com//map...",124000,NaN,4.2,https://www.google.com//maps/place/Cromhouthui...,"(52.368528, 4.8862901)"
12,Bimhuis,Concertzaal of Theater,<WikipediaPage 'Bimhuis'>,https://nl.wikipedia.org/wiki/Bimhuis,17905.0,0.035798,33644.0,3.584943e-03,96,0.063333,...,Oostelijk Havengebied,Oostelijke Handelskade,0,0,"(165000, nan, 4.6, https://www.google.com//map...",165000,NaN,4.6,https://www.google.com//maps/place/Bimhuis/@52...,"(52.3779654, 4.9127861)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Wertheimpark,Park of recreatiegebied,<WikipediaPage 'Wertheimpark'>,https://nl.wikipedia.org/wiki/Wertheimpark,19979.0,0.039950,5191.0,5.531281e-04,55,0.036000,...,Weesperbuurt/Plantage,Plantage,0,14,"(27200, nan, 4.5, https://www.google.com//maps...",27200,NaN,4.5,https://www.google.com//maps/place/Wertheimpar...,"(52.3677165, 4.9087828)"
167,De Hoge Dijk,Park of recreatiegebied,<WikipediaPage 'De Hoge Dijk'>,https://nl.wikipedia.org/wiki/De_Hoge_Dijk,5834.0,0.011633,0.0,0.000000e+00,23,0.014667,...,Amstel III/Bullewijk,Hoge Dijk,0,0,"(1870000, nan, 4.3, https://www.google.com//ma...",1870000,NaN,4.3,https://www.google.com//maps/place/De+Hoge+Dij...,"(52.2864485, 4.9601711)"
168,Siegerpark,Park of recreatiegebied,<WikipediaPage 'Siegerpark'>,https://nl.wikipedia.org/wiki/Siegerpark,7495.0,0.014958,150247.0,1.600960e-02,26,0.016667,...,Sloter-/Riekerpolder,Sloterweg e.o.,0,0,"(6120, nan, 4.4, https://www.google.com//maps/...",6120,NaN,4.4,https://www.google.com//maps/place/Siegerpark/...,"(52.3407925, 4.8190865)"
170,Nieuwmarkt,Markt,<WikipediaPage 'Nieuwmarkt (Amsterdam)'>,https://nl.wikipedia.org/wiki/Nieuwmarkt_(Amst...,43629.0,0.087294,58019.0,6.182226e-03,137,0.090667,...,Nieuwmarkt/Lastage,Nieuwmarkt,0,73,"(1650000, nan, nan, https://www.google.com//ma...",1650000,NaN,NaN,"https://www.google.com//maps/place/Nieuwmarkt,...",NaN


In [105]:
m1 = miss[:20]
m2 = miss[20:40]
m3 = miss[40:60]
m4 = miss[60:80]

In [153]:
m5 = miss[80:100]
m6 = miss[100:]

In [24]:
def get_reviews(monument):
    s = monument.replace(" ", "+")    
    url = 'http://www.google.com/search?q=' + s + "+amsterdam"

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36"}
    page = requests.get(url, headers=headers)        
    soup = BeautifulSoup(page.content, 'html.parser')

    try:
        r_ = soup.find_all("a", {"data-async-trigger":"reviewDialog"})
        reviews = int(re.findall("\d+ ", str(r_[0].find("span")))[0])
    except IndexError:
        reviews = np.nan       
    return reviews

In [ ]:
#get_reviews("Amstelkerk")

In [148]:
m1["g_reviews"] = m1["name"].apply(lambda a: get_reviews(a))

<ipython-input-148-a9dfc0e5449f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m1["g_reviews"] = m1["name"].apply(lambda a: get_reviews(a))


In [150]:
m1

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google,g_results,g_reviews,g_rating,g_link,g_coords
2,Allard Pierson Museum,Museum,<WikipediaPage 'Allard Pierson Museum'>,https://nl.wikipedia.org/wiki/Allard_Pierson_M...,33905.0,0.067828,27502.0,2.930481e-03,109,0.072000,...,Burgwallen-Oude Zijde,BG-terrein e.o.,0,44,"(213000, nan, nan, https://www.google.com//map...",213000,545.0,NaN,https://www.google.com//maps/place/Allard+Pier...,"(52.3687868, 4.8930188)"
3,Amstelkerk,Monument of Gebouw,<WikipediaPage 'Amstelkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amstelkerk_(Amst...,13384.0,0.026747,2.0,2.131104e-07,52,0.034000,...,Grachtengordel-Zuid,Amstelveldbuurt,56,0,"(71300, nan, 4.3, https://www.google.com//maps...",71300,148.0,4.3,https://www.google.com//maps/place/Amstelkerk+...,"(52.3623276, 4.8966565)"
4,Amstelveld,Markt,<WikipediaPage 'Amstelveld'>,https://nl.wikipedia.org/wiki/Amstelveld,20987.0,0.041968,53163.0,5.664794e-03,47,0.030667,...,Grachtengordel-Zuid,Amstelveldbuurt,43,0,"(54800, nan, nan, https://www.google.com//maps...",54800,NaN,NaN,"https://www.google.com//maps/place/Amstelveld,...",NaN
11,Bijbels Museum,Museum,<WikipediaPage 'Bijbels Museum'>,https://nl.wikipedia.org/wiki/Bijbels_Museum,11725.0,0.023426,17952.0,1.912879e-03,81,0.053333,...,Grachtengordel-West,Leidsegracht Noord,85,0,"(124000, nan, 4.2, https://www.google.com//map...",124000,17.0,4.2,https://www.google.com//maps/place/Cromhouthui...,"(52.368528, 4.8862901)"
12,Bimhuis,Concertzaal of Theater,<WikipediaPage 'Bimhuis'>,https://nl.wikipedia.org/wiki/Bimhuis,17905.0,0.035798,33644.0,3.584943e-03,96,0.063333,...,Oostelijk Havengebied,Oostelijke Handelskade,0,0,"(165000, nan, 4.6, https://www.google.com//map...",165000,603.0,4.6,https://www.google.com//maps/place/Bimhuis/@52...,"(52.3779654, 4.9127861)"
13,Blauwbrug,Monument of Gebouw,<WikipediaPage 'Blauwbrug'>,https://nl.wikipedia.org/wiki/Blauwbrug,20136.0,0.040264,21425.0,2.282945e-03,72,0.047333,...,Nieuwmarkt/Lastage,Waterloopleinbuurt,11,8,"(124000, nan, 4.6, https://www.google.com//map...",124000,NaN,4.6,https://www.google.com//maps/dir//Blauwbrug+Br...,NaN
15,Dam,Monument of Gebouw,<WikipediaPage 'Dam (Amsterdam)'>,https://nl.wikipedia.org/wiki/Dam_(Amsterdam),120253.0,0.240687,329892.0,3.515171e-02,273,0.181333,...,Burgwallen-Nieuwe Zijde,Nieuwe Kerk e.o.,0,24,"(38800000, nan, nan, https://www.google.com//m...",38800000,NaN,NaN,"https://www.google.com//maps/place/Dam,+Amster...",NaN
17,De Duif,Monument of Gebouw,<WikipediaPage 'De Duif (kerk)'>,https://nl.wikipedia.org/wiki/De_Duif_(kerk),24398.0,0.048796,0.0,0.000000e+00,104,0.068667,...,De Weteringschans,Utrechtsebuurt Zuid,41,15,"(558000, nan, 4.4, https://www.google.com//map...",558000,571.0,4.4,https://www.google.com//maps/place/De+Duif/@52...,"(52.3614306, 4.8967615)"
18,De Kleine Komedie,Concertzaal of Theater,<WikipediaPage 'De Kleine Komedie'>,https://nl.wikipedia.org/wiki/De_Kleine_Komedie,19117.0,0.038224,7196.0,7.667712e-04,54,0.035333,...,Grachtengordel-Zuid,Rembrandtpleinbuurt,1,71,"(1480000, nan, 4.5, https://www.google.com//ma...",1480000,955.0,4.5,https://www.google.com//maps/place/De+Kleine+K...,"(52.3669133, 4.8959687)"
19,De Krijtberg,Monument of Gebouw,<WikipediaPage 'De Krijtberg'>,https://nl.wikipedia.org/wiki/De_Krijtberg,18049.0,0.036086,21717.0,2.314059e-03,85,0.056000,...,Grachtengordel-Zuid,Leidsegracht Zuid,72,7,"(45900, nan, 4.7, https://www.google.com//maps...",45900,507.0,4.7,https://www.google.com//maps/place/De+Krijtber...,"(52.3677778, 4.8880556)"


In [151]:
m2["g_reviews"] = m2["name"].apply(lambda a: get_reviews(a))

<ipython-input-151-1517a370add9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m2["g_reviews"] = m2["name"].apply(lambda a: get_reviews(a))


In [152]:
m2

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google,g_results,g_reviews,g_rating,g_link,g_coords
45,Noordermarkt,Markt,<WikipediaPage 'Noordermarkt'>,https://nl.wikipedia.org/wiki/Noordermarkt,18146.0,0.036280,25029.0,0.002667,61,0.040000,...,Jordaan,Anjeliersbuurt Noord,48,6,"(234000, nan, nan, https://www.google.com//map...",234000,NaN,NaN,https://www.google.com//maps/place/Noordermark...,NaN
46,Olympisch Stadion,Monument of Gebouw,<WikipediaPage 'Olympisch Stadion (Amsterdam)'>,https://nl.wikipedia.org/wiki/Olympisch_Stadio...,119368.0,0.238915,215010.0,0.022910,435,0.289333,...,Stadionbuurt,Olympisch Stadion e.o.,0,4,"(981000, nan, 4.4, nan, nan)",981000,NaN,4.4,NaN,NaN
47,Oost-Indisch Huis,Monument of Gebouw,<WikipediaPage 'Oost-Indisch Huis (Amsterdam)'>,https://nl.wikipedia.org/wiki/Oost-Indisch_Hui...,37870.0,0.075765,60361.0,0.006432,86,0.056667,...,Burgwallen-Oude Zijde,BG-terrein e.o.,0,85,"(490000, nan, nan, https://www.google.com//map...",490000,28.0,NaN,https://www.google.com//maps/place/Bushuis+-+O...,"(52.3708333, 4.8980556)"
48,Oosterkerk,Monument of Gebouw,<WikipediaPage 'Oosterkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Oosterkerk_(Amst...,13565.0,0.027110,7367.0,0.000785,92,0.060667,...,Oostelijke Eilanden/Kadijken,Wittenburg,0,12,"(87800, nan, 4.2, https://www.google.com//maps...",87800,98.0,4.2,https://www.google.com//maps/place/Oosterkerk/...,"(52.3699949, 4.9193164)"
52,Posthoornkerk,Monument of Gebouw,<WikipediaPage 'Posthoornkerk'>,https://nl.wikipedia.org/wiki/Posthoornkerk,16953.0,0.033892,13104.0,0.001396,35,0.022667,...,Haarlemmerbuurt,Haarlemmerbuurt Oost,59,0,"(18400, nan, 4.5, https://www.google.com//maps...",18400,172.0,4.5,https://www.google.com//maps/place/Posthoornke...,"(52.3812294, 4.8905405)"
53,Pythonbrug,Monument of Gebouw,<WikipediaPage 'Pythonbrug'>,https://nl.wikipedia.org/wiki/Pythonbrug,15201.0,0.030385,34613.0,0.003688,54,0.035333,...,Oostelijk Havengebied,Borneo,0,0,"(9180, nan, 4.6, https://www.google.com//maps/...",9180,304.0,4.6,https://www.google.com//maps/place/Pythonbrug/...,"(52.373207, 4.9488986)"
55,Rembrandtplein,Monument of Gebouw,<WikipediaPage 'Rembrandtplein (Amsterdam)'>,https://nl.wikipedia.org/wiki/Rembrandtplein_(...,52276.0,0.104605,104845.0,0.011172,159,0.105333,...,Grachtengordel-Zuid,Rembrandtpleinbuurt,28,41,"(590000, nan, nan, https://www.google.com//map...",590000,NaN,NaN,https://www.google.com//maps/place/Rembrandtpl...,NaN
56,Rembrandttoren,Monument of Gebouw,<WikipediaPage 'Rembrandttoren'>,https://nl.wikipedia.org/wiki/Rembrandttoren,60104.0,0.120275,50659.0,0.005398,111,0.073333,...,Omval/Overamstel,De Omval,0,0,"(16500, nan, nan, https://www.google.com//maps...",16500,NaN,NaN,https://www.google.com//maps/place/Rembrandtto...,NaN
58,Ronde Lutherse Kerk,Monument of Gebouw,<WikipediaPage 'Ronde Lutherse Kerk'>,https://nl.wikipedia.org/wiki/Ronde_Lutherse_Kerk,18878.0,0.037746,12504.0,0.001332,51,0.033333,...,Burgwallen-Nieuwe Zijde,Hemelrijk,61,15,"(34500, nan, 4.5, https://www.google.com//maps...",34500,4.0,4.5,https://www.google.com//maps/place/Ronde+Luthe...,"(52.3778503, 4.8939605)"
62,Stopera,Monument of Gebouw,<WikipediaPage 'Stopera'>,https://nl.wikipedia.org/wiki/Stopera,72530.0,0.145151,0.0,0.000000,166,0.110000,...,Nieuwmarkt/Lastage,Waterloopleinbuurt,0,1,"(263000, nan, 4.6, https://www.google.com//map...",263000,NaN,4.6,https://www.google.com//maps/dir//Nationale+Op...,NaN


In [159]:
m3["g_reviews"] = m1["name"].apply(lambda a: get_reviews(a))

<ipython-input-159-17446023d801>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m3["g_reviews"] = m1["name"].apply(lambda a: get_reviews(a))


In [160]:
m3

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google,g_results,g_reviews,g_rating,g_link,g_coords
79,Gebouw Plancius,Monument of Gebouw,<WikipediaPage 'Gebouw Plancius'>,https://nl.wikipedia.org/wiki/Gebouw_Plancius,5820.0,0.011605,0.0,0.000000,20,0.012667,...,Weesperbuurt/Plantage,Plantage,0,14,"(9780, nan, 4.0, https://www.google.com//maps/...",9780,NaN,4.0,https://www.google.com//maps/place/Plancius/@5...,"(52.3677817, 4.9127819)"
80,Zuiderbad,Monument of Gebouw,<WikipediaPage 'Zuiderbad'>,https://nl.wikipedia.org/wiki/Zuiderbad,26898.0,0.053801,0.0,0.000000,64,0.042000,...,Museumkwartier,Johannnes Vermeerbuurt,0,6,"(28600, nan, 4.3, https://www.google.com//maps...",28600,NaN,4.3,https://www.google.com//maps/place/Zuiderbad/@...,"(52.3585917, 4.8858944)"
81,Blauwe Theehuis,Monument of Gebouw,<WikipediaPage 'Blauwe Theehuis'>,https://nl.wikipedia.org/wiki/Blauwe_Theehuis,12997.0,0.025972,12708.0,0.001354,34,0.022000,...,Museumkwartier,Vondelpark Oost,0,2,"(73700, nan, 4.4, https://www.google.com//maps...",73700,NaN,4.4,https://www.google.com//maps/place/Proeflokaal...,"(52.3588283, 4.8725491)"
82,Doelen Hotel,Monument of Gebouw,<WikipediaPage 'Doelen Hotel'>,https://nl.wikipedia.org/wiki/Doelen_Hotel,11904.0,0.023784,0.0,0.000000,55,0.036000,...,Burgwallen-Oude Zijde,BG-terrein e.o.,0,28,"(800000, nan, 4.6, https://www.google.com//map...",800000,NaN,4.6,https://www.google.com//maps/place/Hotel+NH+Co...,"(52.36836, 4.89586)"
84,Hollandsche Manege,Monument of Gebouw,<WikipediaPage 'Hollandsche Manege'>,https://nl.wikipedia.org/wiki/Hollandsche_Manege,15170.0,0.030323,12582.0,0.001341,77,0.050667,...,Vondelbuurt,Vondelparkbuurt Midden,0,7,"(28900, nan, 3.7, https://www.google.com//maps...",28900,NaN,3.7,https://www.google.com//maps/place/Hollandsche...,"(52.3614435, 4.8718709)"
85,Huis met de Kabouters,Monument of Gebouw,<WikipediaPage 'Huis met de Kabouters'>,https://nl.wikipedia.org/wiki/Huis_met_de_Kabo...,7997.0,0.015963,5878.0,0.000626,31,0.020000,...,Oude Pijp,Hemonybuurt,0,6,"(80600, nan, 4.3, https://www.google.com//maps...",80600,NaN,4.3,https://www.google.com//maps/place/Huis+met+de...,"(52.3559294, 4.9042974)"
86,Kerk van Durgerdam,Monument of Gebouw,<WikipediaPage 'Kerk van Durgerdam'>,https://nl.wikipedia.org/wiki/Kerk_van_Durgerdam,3779.0,0.007519,9384807.0,1.000000,8,0.004667,...,Waterland,Durgerdam,0,2,"(44100, nan, 4.6, https://www.google.com//maps...",44100,NaN,4.6,https://www.google.com//maps/place/Kerk+van+Du...,"(52.3783681, 4.9882043)"
88,Hollandsche Schouwburg,Herdenkings Monument,<WikipediaPage 'Hollandsche Schouwburg'>,https://nl.wikipedia.org/wiki/Hollandsche_Scho...,56304.0,0.112668,19800.0,0.002110,180,0.119333,...,Weesperbuurt/Plantage,Plantage,0,16,"(95200, nan, 4.4, https://www.google.com//maps...",95200,NaN,4.4,https://www.google.com//maps/place/Hollandsche...,"(52.366467, 4.9111234)"
89,American Hotel,Monument of Gebouw,<WikipediaPage 'American Hotel'>,https://nl.wikipedia.org/wiki/American_Hotel,37073.0,0.074170,0.0,0.000000,105,0.069333,...,De Weteringschans,Leidsebuurt Zuidwest,0,6,"(59500000, nan, 4.3, https://www.google.com//m...",59500000,NaN,4.3,https://www.google.com//maps/place/Hard+Rock+H...,"(52.363889, 4.881302)"
90,Jeugdtheater De Krakeling,Monument of Gebouw,<WikipediaPage 'Jeugdtheater De Krakeling'>,https://nl.wikipedia.org/wiki/Jeugdtheater_De_...,5068.0,0.010100,2296.0,0.000245,31,0.020000,...,Spaarndammer- en Zeeheldenbuurt,Westergasfabriek,0,6,"(19800, nan, 4.4, https://www.google.com//maps...",19800,NaN,4.4,https://www.google.com//maps/place/Theater+De+...,"(52.3861832, 4.8735528)"


In [161]:
m4["g_reviews"] = m4["name"].apply(lambda a: get_reviews(a))

<ipython-input-161-ce0103d1f415>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m4["g_reviews"] = m4["name"].apply(lambda a: get_reviews(a))


In [162]:
m4

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google,g_results,g_reviews,g_rating,g_link,g_coords
103,Aluminiumbrug,Monument of Gebouw,<WikipediaPage 'Aluminiumbrug'>,https://nl.wikipedia.org/wiki/Aluminiumbrug,2933.0,0.005825,0.0,0.000000e+00,13,0.008000,...,Burgwallen-Oude Zijde,BG-terrein e.o.,0,58,"(7090, nan, nan, https://www.google.com//maps/...",7090,NaN,NaN,https://www.google.com//maps/place/Aluminiumbr...,NaN
104,Westergasfabriek,Monument of Gebouw,<WikipediaPage 'Westergasfabriek'>,https://nl.wikipedia.org/wiki/Westergasfabriek,72796.0,0.145683,24039.0,2.561480e-03,127,0.084000,...,Spaarndammer- en Zeeheldenbuurt,Westergasfabriek,0,4,"(267000, nan, 4.3, https://www.google.com//map...",267000,443.0,4.3,https://www.google.com//maps/place/Zuiveringsh...,"(52.3860227, 4.8721395)"
105,Museumbrug,Monument of Gebouw,<WikipediaPage 'Museumbrug (Amsterdam)'>,https://nl.wikipedia.org/wiki/Museumbrug_(Amst...,2327.0,0.004612,0.0,0.000000e+00,18,0.011333,...,De Weteringschans,Den Texbuurt,0,9,"(21400, nan, nan, https://www.google.com//maps...",21400,NaN,NaN,"https://www.google.com//maps/place/Museumbrug,...",NaN
106,P.L. Kramerbrug,Monument of Gebouw,<WikipediaPage 'P.L. Kramerbrug'>,https://nl.wikipedia.org/wiki/P.L._Kramerbrug,4338.0,0.008638,0.0,0.000000e+00,29,0.018667,...,Zuid Pijp,Diamantbuurt,0,6,"(3770, nan, nan, https://www.google.com//maps/...",3770,NaN,NaN,https://www.google.com//maps/place/P.L.+Kramer...,NaN
107,Amstel Hotel,Monument of Gebouw,<WikipediaPage 'Amstel Hotel'>,https://nl.wikipedia.org/wiki/Amstel_Hotel,67009.0,0.134098,34650.0,3.692138e-03,170,0.112667,...,Weesperbuurt/Plantage,Sarphatistrook,0,3,"(3790000, nan, 4.6, https://www.google.com//ma...",3790000,NaN,4.6,https://www.google.com//maps/place/InterContin...,"(52.3600068, 4.9051643)"
108,Nieuwer-Amstel,Monument of Gebouw,<WikipediaPage 'Nieuwer-Amstel'>,https://nl.wikipedia.org/wiki/Nieuwer-Amstel,32638.0,0.065292,349522.0,3.724339e-02,113,0.074667,...,Nieuwe Pijp,Willibrordusbuurt,0,4,"(4590000, nan, nan, https://www.google.com//ma...",4590000,NaN,NaN,https://www.google.com//maps/dir/Amstelveen/Am...,NaN
109,Rijksmuseumtuinen,Monument of Gebouw,<WikipediaPage 'Rijksmuseumtuinen'>,https://nl.wikipedia.org/wiki/Rijksmuseumtuinen,5216.0,0.010396,0.0,0.000000e+00,46,0.030000,...,Museumkwartier,Museumplein,0,3,"(5210, nan, 4.7, https://www.google.com//maps/...",5210,651.0,4.7,https://www.google.com//maps/place/Rijksmuseum...,"(52.3592285, 4.8851334)"
110,Waalse Kerk,Monument of Gebouw,<WikipediaPage 'Waalse Kerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Waalse_Kerk_(Ams...,16013.0,0.032010,14576.0,1.553149e-03,111,0.073333,...,Burgwallen-Oude Zijde,BG-terrein e.o.,0,87,"(204000, nan, 4.4, https://www.google.com//map...",204000,128.0,4.4,https://www.google.com//maps/place/De+Waalse+K...,"(52.371041, 4.897241)"
111,Oudemanhuispoort,Markt,<WikipediaPage 'Oudemanhuispoort'>,https://nl.wikipedia.org/wiki/Oudemanhuispoort,25509.0,0.051020,503825.0,5.368517e-02,87,0.057333,...,Burgwallen-Oude Zijde,BG-terrein e.o.,0,42,"(37700, nan, nan, https://www.google.com//maps...",37700,NaN,NaN,https://www.google.com//maps/place/Oudemanhuis...,NaN
112,Obrechtkerk,Monument of Gebouw,<WikipediaPage 'Obrechtkerk'>,https://nl.wikipedia.org/wiki/Obrechtkerk,10714.0,0.021402,2975.0,3.170017e-04,43,0.028000,...,Museumkwartier,Cornelis Schuytbuurt,0,2,"(6550, nan, 4.5, https://www.google.com//maps/...",6550,61.0,4.5,https://www.google.com//maps/place/Obrechtkerk...,"(52.3558019, 4.8753696)"


In [168]:
m5["g_reviews"] = m5["name"].apply(lambda a: get_reviews(a))

<ipython-input-168-9e5b27ed50f1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m5["g_reviews"] = m5["name"].apply(lambda a: get_reviews(a))


In [169]:
m5

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google,g_results,g_reviews,g_rating,g_link,g_coords
124,Auschwitzmonument,Herdenkings Monument,<WikipediaPage 'Auschwitzmonument (Amsterdam)'>,https://nl.wikipedia.org/wiki/Auschwitzmonumen...,39822.0,0.079673,0.0,0.000000,93,0.061333,...,Weesperbuurt/Plantage,Plantage,0,13,"(9100, nan, 4.4, https://www.google.com//maps/...",9100,458.0,4.4,https://www.google.com//maps/place/Auschwitzmo...,"(52.3679582, 4.9092116)"
125,De Dokwerker,Herdenkings Monument,<WikipediaPage 'De Dokwerker'>,https://nl.wikipedia.org/wiki/De_Dokwerker,52673.0,0.105399,75974.0,0.008095,111,0.073333,...,Nieuwmarkt/Lastage,Waterloopleinbuurt,6,17,"(34500, nan, 4.4, https://www.google.com//maps...",34500,74.0,4.4,https://www.google.com//maps/place/De+Dokwerke...,"(52.3672213, 4.9049742)"
126,Nationaal monument slavernijverleden,Herdenkings Monument,<WikipediaPage 'Nationaal monument slavernijve...,https://nl.wikipedia.org/wiki/Nationaal_monume...,23252.0,0.046502,0.0,0.000000,78,0.051333,...,Oosterparkbuurt,Oosterpark,0,0,"(27200, nan, 4.4, https://www.google.com//maps...",27200,72.0,4.4,https://www.google.com//maps/place/Nationaal+S...,"(52.3586624, 4.9178383)"
127,Museum Willet-Holthuysen,Museum,<WikipediaPage 'Museum Willet-Holthuysen'>,https://nl.wikipedia.org/wiki/Museum_Willet-Ho...,18868.0,0.037726,26002.0,0.002771,59,0.038667,...,Grachtengordel-Zuid,Rembrandtpleinbuurt,39,4,"(69200, nan, 4.5, https://www.google.com//maps...",69200,754.0,4.5,https://www.google.com//maps/place/Museum+Will...,"(52.3656739, 4.8989802)"
129,Huis Marseille,Museum,<WikipediaPage 'Huis Marseille'>,https://nl.wikipedia.org/wiki/Huis_Marseille,11996.0,0.023969,8051.0,0.000858,56,0.036667,...,Grachtengordel-West,Leidsegracht Noord,73,0,"(2640000, nan, 4.3, https://www.google.com//ma...",2640000,918.0,4.3,https://www.google.com//maps/place/Huis+Marsei...,"(52.3676378, 4.8848239)"
130,Nederlands Uitvaart Museum Tot Zover,Museum,<WikipediaPage 'Nederlands Uitvaart Museum Tot...,https://nl.wikipedia.org/wiki/Nederlands_Uitva...,5267.0,0.010498,7557.0,0.000805,32,0.020667,...,Betondorp,Nieuwe Oosterbegraafplaats,0,24,"(43700, nan, 4.2, https://www.google.com//maps...",43700,196.0,4.2,https://www.google.com//maps/place/Museum+Tot+...,"(52.3463014, 4.9386049)"
134,Amsterdam Tulip Museum,Museum,<WikipediaPage 'Amsterdam Tulip Museum'>,https://nl.wikipedia.org/wiki/Amsterdam_Tulip_...,2823.0,0.005605,1949948.0,0.207777,24,0.015333,...,Grachtengordel-West,Leliegracht e.o.,60,0,"(2040000, nan, 4.1, https://www.google.com//ma...",2040000,915.0,4.1,https://www.google.com//maps/place/Amsterdam+T...,"(52.3763222, 4.8841231)"
135,Molen van Sloten,Museum,<WikipediaPage 'Molen van Sloten'>,https://nl.wikipedia.org/wiki/Molen_van_Sloten,9928.0,0.019829,36117.0,0.003848,77,0.050667,...,Sloter-/Riekerpolder,Dorp Sloten,0,0,"(201000, nan, 4.5, https://www.google.com//map...",201000,547.0,4.5,https://www.google.com//maps/place/Molen+van+S...,"(52.3415626, 4.7922238)"
136,Van Eesteren Museum,Museum,<WikipediaPage 'Van Eesteren Museum'>,https://nl.wikipedia.org/wiki/Van_Eesteren_Museum,4582.0,0.009127,16280.0,0.001735,125,0.082667,...,Slotermeer-Noordoost,Buurt 2,0,0,"(48300, nan, 4.2, https://www.google.com//maps...",48300,104.0,4.2,https://www.google.com//maps/place/Van+Eestere...,"(52.3728614, 4.8262597)"
137,Museum Amsterdam-Noord,Museum,<WikipediaPage 'Museum Amsterdam-Noord'>,https://nl.wikipedia.org/wiki/Museum_Amsterdam...,2932.0,0.005823,63068.0,0.006720,26,0.016667,...,IJplein/Vogelbuurt,Vliegenbos,0,0,"(192000000, nan, nan, nan, nan)",192000000,NaN,NaN,NaN,NaN


In [170]:
m6["g_reviews"] = m6["name"].apply(lambda a: get_reviews(a))

<ipython-input-170-39ff36f066ae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m6["g_reviews"] = m6["name"].apply(lambda a: get_reviews(a))


In [171]:
m6

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google,g_results,g_reviews,g_rating,g_link,g_coords
151,Erasmuspark,Park of recreatiegebied,<WikipediaPage 'Erasmuspark'>,https://nl.wikipedia.org/wiki/Erasmuspark,13061.0,0.026101,22458.0,0.002393,66,0.043333,...,Erasmuspark,Erasmusparkbuurt West,0,0,"(86900, nan, nan, https://www.google.com//maps...",86900,NaN,NaN,https://www.google.com//maps/place/Erasmuspark...,NaN
152,Flevopark,Park of recreatiegebied,<WikipediaPage 'Flevopark'>,https://nl.wikipedia.org/wiki/Flevopark,16338.0,0.032661,15547.0,0.001657,50,0.032667,...,Indische Buurt Oost,Flevopark,0,2,"(83700, nan, 4.6, https://www.google.com//maps...",83700,619.0,4.6,https://www.google.com//maps/place/Flevopark/@...,"(52.3586371, 4.948333)"
153,Gaasperplas,Park of recreatiegebied,<WikipediaPage 'Gaasperplas (water)'>,https://nl.wikipedia.org/wiki/Gaasperplas_(water),16609.0,0.033203,12224.0,0.001303,52,0.034000,...,Nellestein,Gaasperpark,0,0,"(128000, nan, 4.5, nan, nan)",128000,NaN,4.5,NaN,NaN
157,Martin Luther Kingpark,Park of recreatiegebied,<WikipediaPage 'Martin Luther Kingpark'>,https://nl.wikipedia.org/wiki/Martin_Luther_Ki...,6575.0,0.013116,0.0,0.000000,28,0.018000,...,Rijnbuurt,Rijnbuurt Midden,0,0,"(59000, nan, 4.3, https://www.google.com//maps...",59000,872.0,4.3,https://www.google.com//maps/place/Martin+Luth...,"(52.3403749, 4.9102053)"
158,Tuinen van West,Park of recreatiegebied,<WikipediaPage 'Tuinen van West'>,https://nl.wikipedia.org/wiki/Tuinen_van_West,6312.0,0.012590,0.0,0.000000,18,0.011333,...,Lutkemeer/Ookmeer,Osdorper Bovenpolder,0,0,"(949000, nan, 4.5, https://www.google.com//map...",949000,54.0,4.5,https://www.google.com//maps/place/Tuinen+Van+...,"(52.378054, 4.7797082)"
160,Sarphatipark,Park of recreatiegebied,<WikipediaPage 'Sarphatipark'>,https://nl.wikipedia.org/wiki/Sarphatipark,25020.0,0.050041,23766.0,0.002532,70,0.046000,...,Oude Pijp,Sarphatiparkbuurt,0,14,"(134000, nan, nan, https://www.google.com//map...",134000,NaN,NaN,https://www.google.com//maps/place/Sarphatipar...,NaN
161,Vliegenbos,Park of recreatiegebied,<WikipediaPage 'Vliegenbos'>,https://nl.wikipedia.org/wiki/Vliegenbos,16650.0,0.033285,63068.0,0.006720,81,0.053333,...,IJplein/Vogelbuurt,Vliegenbos,0,0,"(25900, nan, 4.4, nan, nan)",25900,NaN,4.4,NaN,NaN
164,Sloterplas,Park of recreatiegebied,<WikipediaPage 'Sloterplas'>,https://nl.wikipedia.org/wiki/Sloterplas,26669.0,0.053342,362439.0,0.038620,103,0.068000,...,Slotermeer-Zuidwest,Sloterpark,0,0,"(129000, nan, 4.5, nan, nan)",129000,NaN,4.5,NaN,NaN
165,Wertheimpark,Park of recreatiegebied,<WikipediaPage 'Wertheimpark'>,https://nl.wikipedia.org/wiki/Wertheimpark,19979.0,0.039950,5191.0,0.000553,55,0.036000,...,Weesperbuurt/Plantage,Plantage,0,14,"(27200, nan, 4.5, https://www.google.com//maps...",27200,182.0,4.5,https://www.google.com//maps/place/Wertheimpar...,"(52.3677165, 4.9087828)"
167,De Hoge Dijk,Park of recreatiegebied,<WikipediaPage 'De Hoge Dijk'>,https://nl.wikipedia.org/wiki/De_Hoge_Dijk,5834.0,0.011633,0.0,0.000000,23,0.014667,...,Amstel III/Bullewijk,Hoge Dijk,0,0,"(1870000, nan, 4.3, https://www.google.com//ma...",1870000,563.0,4.3,https://www.google.com//maps/place/De+Hoge+Dij...,"(52.2864485, 4.9601711)"


In [172]:
m_final = pd.concat([m1,m2,m3,m4,m5,m6])
m_final

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google,g_results,g_reviews,g_rating,g_link,g_coords
2,Allard Pierson Museum,Museum,<WikipediaPage 'Allard Pierson Museum'>,https://nl.wikipedia.org/wiki/Allard_Pierson_M...,33905.0,0.067828,27502.0,2.930481e-03,109,0.072000,...,Burgwallen-Oude Zijde,BG-terrein e.o.,0,44,"(213000, nan, nan, https://www.google.com//map...",213000,545.0,NaN,https://www.google.com//maps/place/Allard+Pier...,"(52.3687868, 4.8930188)"
3,Amstelkerk,Monument of Gebouw,<WikipediaPage 'Amstelkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amstelkerk_(Amst...,13384.0,0.026747,2.0,2.131104e-07,52,0.034000,...,Grachtengordel-Zuid,Amstelveldbuurt,56,0,"(71300, nan, 4.3, https://www.google.com//maps...",71300,148.0,4.3,https://www.google.com//maps/place/Amstelkerk+...,"(52.3623276, 4.8966565)"
4,Amstelveld,Markt,<WikipediaPage 'Amstelveld'>,https://nl.wikipedia.org/wiki/Amstelveld,20987.0,0.041968,53163.0,5.664794e-03,47,0.030667,...,Grachtengordel-Zuid,Amstelveldbuurt,43,0,"(54800, nan, nan, https://www.google.com//maps...",54800,NaN,NaN,"https://www.google.com//maps/place/Amstelveld,...",NaN
11,Bijbels Museum,Museum,<WikipediaPage 'Bijbels Museum'>,https://nl.wikipedia.org/wiki/Bijbels_Museum,11725.0,0.023426,17952.0,1.912879e-03,81,0.053333,...,Grachtengordel-West,Leidsegracht Noord,85,0,"(124000, nan, 4.2, https://www.google.com//map...",124000,17.0,4.2,https://www.google.com//maps/place/Cromhouthui...,"(52.368528, 4.8862901)"
12,Bimhuis,Concertzaal of Theater,<WikipediaPage 'Bimhuis'>,https://nl.wikipedia.org/wiki/Bimhuis,17905.0,0.035798,33644.0,3.584943e-03,96,0.063333,...,Oostelijk Havengebied,Oostelijke Handelskade,0,0,"(165000, nan, 4.6, https://www.google.com//map...",165000,603.0,4.6,https://www.google.com//maps/place/Bimhuis/@52...,"(52.3779654, 4.9127861)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Wertheimpark,Park of recreatiegebied,<WikipediaPage 'Wertheimpark'>,https://nl.wikipedia.org/wiki/Wertheimpark,19979.0,0.039950,5191.0,5.531281e-04,55,0.036000,...,Weesperbuurt/Plantage,Plantage,0,14,"(27200, nan, 4.5, https://www.google.com//maps...",27200,182.0,4.5,https://www.google.com//maps/place/Wertheimpar...,"(52.3677165, 4.9087828)"
167,De Hoge Dijk,Park of recreatiegebied,<WikipediaPage 'De Hoge Dijk'>,https://nl.wikipedia.org/wiki/De_Hoge_Dijk,5834.0,0.011633,0.0,0.000000e+00,23,0.014667,...,Amstel III/Bullewijk,Hoge Dijk,0,0,"(1870000, nan, 4.3, https://www.google.com//ma...",1870000,563.0,4.3,https://www.google.com//maps/place/De+Hoge+Dij...,"(52.2864485, 4.9601711)"
168,Siegerpark,Park of recreatiegebied,<WikipediaPage 'Siegerpark'>,https://nl.wikipedia.org/wiki/Siegerpark,7495.0,0.014958,150247.0,1.600960e-02,26,0.016667,...,Sloter-/Riekerpolder,Sloterweg e.o.,0,0,"(6120, nan, 4.4, https://www.google.com//maps/...",6120,97.0,4.4,https://www.google.com//maps/place/Siegerpark/...,"(52.3407925, 4.8190865)"
170,Nieuwmarkt,Markt,<WikipediaPage 'Nieuwmarkt (Amsterdam)'>,https://nl.wikipedia.org/wiki/Nieuwmarkt_(Amst...,43629.0,0.087294,58019.0,6.182226e-03,137,0.090667,...,Nieuwmarkt/Lastage,Nieuwmarkt,0,73,"(1650000, nan, nan, https://www.google.com//ma...",1650000,NaN,NaN,"https://www.google.com//maps/place/Nieuwmarkt,...",NaN


In [173]:
final.isna().sum()/len(final)

name                         0.000000
label                        0.000000
wiki_page_obj                0.000000
wiki_url                     0.000000
wiki_nl_views                0.000000
wiki_norm_nl_views           0.000000
wiki_eng_views               0.000000
wiki_norm_eng_views          0.000000
wiki_nr_of_edits             0.000000
wiki_norm_nr_of_edits        0.000000
wiki_nr_of_languages         0.000000
wiki_norm_nr_of_languages    0.000000
TA_name                      0.000000
TA_rating                    0.000000
TA_reviews                   0.000000
TA_url                       0.000000
wiki_coordinates             0.000000
WKT_LNG_LAT                  0.000000
District                     0.005780
Neighborhoodcombination      0.005780
Neighborhood                 0.005780
UNESCO                       0.000000
Monuments                    0.000000
google                       0.000000
g_results                    0.000000
g_reviews                    0.653179
g_rating    

In [199]:
for i in m_final.index:
#     print(i)
#     try:
    final.at[i,"g_reviews"] = m_final.at[i,"g_reviews"]
#     except IndexError:
#         final.at[2,"g_reviews"] = np.nan

In [187]:
m_final

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google,g_results,g_reviews,g_rating,g_link,g_coords
2,Allard Pierson Museum,Museum,<WikipediaPage 'Allard Pierson Museum'>,https://nl.wikipedia.org/wiki/Allard_Pierson_M...,33905.0,0.067828,27502.0,2.930481e-03,109,0.072000,...,Burgwallen-Oude Zijde,BG-terrein e.o.,0,44,"(213000, nan, nan, https://www.google.com//map...",213000,545.0,NaN,https://www.google.com//maps/place/Allard+Pier...,"(52.3687868, 4.8930188)"
3,Amstelkerk,Monument of Gebouw,<WikipediaPage 'Amstelkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amstelkerk_(Amst...,13384.0,0.026747,2.0,2.131104e-07,52,0.034000,...,Grachtengordel-Zuid,Amstelveldbuurt,56,0,"(71300, nan, 4.3, https://www.google.com//maps...",71300,148.0,4.3,https://www.google.com//maps/place/Amstelkerk+...,"(52.3623276, 4.8966565)"
4,Amstelveld,Markt,<WikipediaPage 'Amstelveld'>,https://nl.wikipedia.org/wiki/Amstelveld,20987.0,0.041968,53163.0,5.664794e-03,47,0.030667,...,Grachtengordel-Zuid,Amstelveldbuurt,43,0,"(54800, nan, nan, https://www.google.com//maps...",54800,NaN,NaN,"https://www.google.com//maps/place/Amstelveld,...",NaN
11,Bijbels Museum,Museum,<WikipediaPage 'Bijbels Museum'>,https://nl.wikipedia.org/wiki/Bijbels_Museum,11725.0,0.023426,17952.0,1.912879e-03,81,0.053333,...,Grachtengordel-West,Leidsegracht Noord,85,0,"(124000, nan, 4.2, https://www.google.com//map...",124000,17.0,4.2,https://www.google.com//maps/place/Cromhouthui...,"(52.368528, 4.8862901)"
12,Bimhuis,Concertzaal of Theater,<WikipediaPage 'Bimhuis'>,https://nl.wikipedia.org/wiki/Bimhuis,17905.0,0.035798,33644.0,3.584943e-03,96,0.063333,...,Oostelijk Havengebied,Oostelijke Handelskade,0,0,"(165000, nan, 4.6, https://www.google.com//map...",165000,603.0,4.6,https://www.google.com//maps/place/Bimhuis/@52...,"(52.3779654, 4.9127861)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,Wertheimpark,Park of recreatiegebied,<WikipediaPage 'Wertheimpark'>,https://nl.wikipedia.org/wiki/Wertheimpark,19979.0,0.039950,5191.0,5.531281e-04,55,0.036000,...,Weesperbuurt/Plantage,Plantage,0,14,"(27200, nan, 4.5, https://www.google.com//maps...",27200,182.0,4.5,https://www.google.com//maps/place/Wertheimpar...,"(52.3677165, 4.9087828)"
167,De Hoge Dijk,Park of recreatiegebied,<WikipediaPage 'De Hoge Dijk'>,https://nl.wikipedia.org/wiki/De_Hoge_Dijk,5834.0,0.011633,0.0,0.000000e+00,23,0.014667,...,Amstel III/Bullewijk,Hoge Dijk,0,0,"(1870000, nan, 4.3, https://www.google.com//ma...",1870000,563.0,4.3,https://www.google.com//maps/place/De+Hoge+Dij...,"(52.2864485, 4.9601711)"
168,Siegerpark,Park of recreatiegebied,<WikipediaPage 'Siegerpark'>,https://nl.wikipedia.org/wiki/Siegerpark,7495.0,0.014958,150247.0,1.600960e-02,26,0.016667,...,Sloter-/Riekerpolder,Sloterweg e.o.,0,0,"(6120, nan, 4.4, https://www.google.com//maps/...",6120,97.0,4.4,https://www.google.com//maps/place/Siegerpark/...,"(52.3407925, 4.8190865)"
170,Nieuwmarkt,Markt,<WikipediaPage 'Nieuwmarkt (Amsterdam)'>,https://nl.wikipedia.org/wiki/Nieuwmarkt_(Amst...,43629.0,0.087294,58019.0,6.182226e-03,137,0.090667,...,Nieuwmarkt/Lastage,Nieuwmarkt,0,73,"(1650000, nan, nan, https://www.google.com//ma...",1650000,NaN,NaN,"https://www.google.com//maps/place/Nieuwmarkt,...",NaN


In [198]:
final.at[2,"g_reviews"]=545.0
final.at[2,"g_reviews"]

545.0

In [200]:
final

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google,g_results,g_reviews,g_rating,g_link,g_coords
0,AFAS Live,Monument of Gebouw,<WikipediaPage 'AFAS Live'>,https://nl.wikipedia.org/wiki/AFAS_Live,124539.0,0.249267,105429.0,1.123401e-02,45,0.029333,...,Amstel III/Bullewijk,Hoofdcentrum Zuidoost,0,0,"(2070000, 9285, 4.3, https://www.google.com//m...",2070000,9285.0,4.3,https://www.google.com//maps/place/AFAS+Live/@...,"(52.3119125, 4.9445428)"
1,Albert Cuypmarkt,Markt,<WikipediaPage 'Albert Cuypmarkt'>,https://nl.wikipedia.org/wiki/Albert_Cuypmarkt,58069.0,0.116201,110365.0,1.175996e-02,70,0.046000,...,Oude Pijp,Sarphatiparkbuurt,0,3,"(109000, 25906, 4.3, https://www.google.com//m...",109000,25906.0,4.3,https://www.google.com//maps/place/Albert+Cuyp...,"(52.3559673, 4.8953871)"
2,Allard Pierson Museum,Museum,<WikipediaPage 'Allard Pierson Museum'>,https://nl.wikipedia.org/wiki/Allard_Pierson_M...,33905.0,0.067828,27502.0,2.930481e-03,109,0.072000,...,Burgwallen-Oude Zijde,BG-terrein e.o.,0,44,"(213000, nan, nan, https://www.google.com//map...",213000,545.0,NaN,https://www.google.com//maps/place/Allard+Pier...,"(52.3687868, 4.8930188)"
3,Amstelkerk,Monument of Gebouw,<WikipediaPage 'Amstelkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amstelkerk_(Amst...,13384.0,0.026747,2.0,2.131104e-07,52,0.034000,...,Grachtengordel-Zuid,Amstelveldbuurt,56,0,"(71300, nan, 4.3, https://www.google.com//maps...",71300,148.0,4.3,https://www.google.com//maps/place/Amstelkerk+...,"(52.3623276, 4.8966565)"
4,Amstelveld,Markt,<WikipediaPage 'Amstelveld'>,https://nl.wikipedia.org/wiki/Amstelveld,20987.0,0.041968,53163.0,5.664794e-03,47,0.030667,...,Grachtengordel-Zuid,Amstelveldbuurt,43,0,"(54800, nan, nan, https://www.google.com//maps...",54800,NaN,NaN,"https://www.google.com//maps/place/Amstelveld,...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,Siegerpark,Park of recreatiegebied,<WikipediaPage 'Siegerpark'>,https://nl.wikipedia.org/wiki/Siegerpark,7495.0,0.014958,150247.0,1.600960e-02,26,0.016667,...,Sloter-/Riekerpolder,Sloterweg e.o.,0,0,"(6120, nan, 4.4, https://www.google.com//maps/...",6120,97.0,4.4,https://www.google.com//maps/place/Siegerpark/...,"(52.3407925, 4.8190865)"
169,Noorderpark,Park of recreatiegebied,<WikipediaPage 'Noorderpark (Amsterdam)'>,https://nl.wikipedia.org/wiki/Noorderpark_(Ams...,8852.0,0.017675,0.0,0.000000e+00,37,0.024000,...,Volewijck,Bloemenbuurt Zuid,0,1,"(84400, 2320, 4.4, https://www.google.com//map...",84400,2320.0,4.4,https://www.google.com//maps/place/Noorderpark...,"(52.3958556, 4.9200389)"
170,Nieuwmarkt,Markt,<WikipediaPage 'Nieuwmarkt (Amsterdam)'>,https://nl.wikipedia.org/wiki/Nieuwmarkt_(Amst...,43629.0,0.087294,58019.0,6.182226e-03,137,0.090667,...,Nieuwmarkt/Lastage,Nieuwmarkt,0,73,"(1650000, nan, nan, https://www.google.com//ma...",1650000,NaN,NaN,"https://www.google.com//maps/place/Nieuwmarkt,...",NaN
171,Amsterdamse Poort,Markt,<WikipediaPage 'Amsterdamse Poort (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amsterdamse_Poor...,17723.0,0.035433,0.0,0.000000e+00,54,0.035333,...,"Bijlmer Centrum (D,F,H)",Amsterdamse Poort,0,0,"(328000, 6956, 4.1, https://www.google.com//ma...",328000,6956.0,4.1,https://www.google.com//maps/place/Amsterdamse...,"(52.3140525, 4.9527714)"


In [201]:
final.isna().sum()/len(final)

name                         0.000000
label                        0.000000
wiki_page_obj                0.000000
wiki_url                     0.000000
wiki_nl_views                0.000000
wiki_norm_nl_views           0.000000
wiki_eng_views               0.000000
wiki_norm_eng_views          0.000000
wiki_nr_of_edits             0.000000
wiki_norm_nr_of_edits        0.000000
wiki_nr_of_languages         0.000000
wiki_norm_nr_of_languages    0.000000
TA_name                      0.000000
TA_rating                    0.000000
TA_reviews                   0.000000
TA_url                       0.000000
wiki_coordinates             0.000000
WKT_LNG_LAT                  0.000000
District                     0.005780
Neighborhoodcombination      0.005780
Neighborhood                 0.005780
UNESCO                       0.000000
Monuments                    0.000000
google                       0.000000
g_results                    0.000000
g_reviews                    0.323699
g_rating    

In [202]:
final.to_csv("fulldata_draft3.csv", index=False)

In [205]:
final["g_norm_results"] = (final["g_results"]-final["g_results"].min())/(final["g_results"].max()-final["g_results"].min())
final["g_norm_reviews"] = (final["g_reviews"]-final["g_reviews"].min())/(final["g_reviews"].max()-final["g_reviews"].min())
final["g_norm_rating"] = (final["g_rating"]-final["g_rating"].min())/(final["g_rating"].max()-final["g_rating"].min())

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Monuments,google,g_results,g_reviews,g_rating,g_link,g_coords,g_norm_results,g_norm_reviews,g_norm_rating
0,AFAS Live,Monument of Gebouw,<WikipediaPage 'AFAS Live'>,https://nl.wikipedia.org/wiki/AFAS_Live,124539.0,0.249267,105429.0,1.123401e-02,45,0.029333,...,0,"(2070000, 9285, 4.3, https://www.google.com//m...",2070000,9285.0,4.3,https://www.google.com//maps/place/AFAS+Live/@...,"(52.3119125, 4.9445428)",0.010771,0.141388,0.533333
1,Albert Cuypmarkt,Markt,<WikipediaPage 'Albert Cuypmarkt'>,https://nl.wikipedia.org/wiki/Albert_Cuypmarkt,58069.0,0.116201,110365.0,1.175996e-02,70,0.046000,...,3,"(109000, 25906, 4.3, https://www.google.com//m...",109000,25906.0,4.3,https://www.google.com//maps/place/Albert+Cuyp...,"(52.3559673, 4.8953871)",0.000558,0.394595,0.533333
2,Allard Pierson Museum,Museum,<WikipediaPage 'Allard Pierson Museum'>,https://nl.wikipedia.org/wiki/Allard_Pierson_M...,33905.0,0.067828,27502.0,2.930481e-03,109,0.072000,...,44,"(213000, nan, nan, https://www.google.com//map...",213000,545.0,NaN,https://www.google.com//maps/place/Allard+Pier...,"(52.3687868, 4.8930188)",0.001099,0.008242,NaN
3,Amstelkerk,Monument of Gebouw,<WikipediaPage 'Amstelkerk (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amstelkerk_(Amst...,13384.0,0.026747,2.0,2.131104e-07,52,0.034000,...,0,"(71300, nan, 4.3, https://www.google.com//maps...",71300,148.0,4.3,https://www.google.com//maps/place/Amstelkerk+...,"(52.3623276, 4.8966565)",0.000361,0.002194,0.533333
4,Amstelveld,Markt,<WikipediaPage 'Amstelveld'>,https://nl.wikipedia.org/wiki/Amstelveld,20987.0,0.041968,53163.0,5.664794e-03,47,0.030667,...,0,"(54800, nan, nan, https://www.google.com//maps...",54800,NaN,NaN,"https://www.google.com//maps/place/Amstelveld,...",NaN,0.000276,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,Siegerpark,Park of recreatiegebied,<WikipediaPage 'Siegerpark'>,https://nl.wikipedia.org/wiki/Siegerpark,7495.0,0.014958,150247.0,1.600960e-02,26,0.016667,...,0,"(6120, nan, 4.4, https://www.google.com//maps/...",6120,97.0,4.4,https://www.google.com//maps/place/Siegerpark/...,"(52.3407925, 4.8190865)",0.000022,0.001417,0.600000
169,Noorderpark,Park of recreatiegebied,<WikipediaPage 'Noorderpark (Amsterdam)'>,https://nl.wikipedia.org/wiki/Noorderpark_(Ams...,8852.0,0.017675,0.0,0.000000e+00,37,0.024000,...,1,"(84400, 2320, 4.4, https://www.google.com//map...",84400,2320.0,4.4,https://www.google.com//maps/place/Noorderpark...,"(52.3958556, 4.9200389)",0.000430,0.035282,0.600000
170,Nieuwmarkt,Markt,<WikipediaPage 'Nieuwmarkt (Amsterdam)'>,https://nl.wikipedia.org/wiki/Nieuwmarkt_(Amst...,43629.0,0.087294,58019.0,6.182226e-03,137,0.090667,...,73,"(1650000, nan, nan, https://www.google.com//ma...",1650000,NaN,NaN,"https://www.google.com//maps/place/Nieuwmarkt,...",NaN,0.008584,NaN,NaN
171,Amsterdamse Poort,Markt,<WikipediaPage 'Amsterdamse Poort (Amsterdam)'>,https://nl.wikipedia.org/wiki/Amsterdamse_Poor...,17723.0,0.035433,0.0,0.000000e+00,54,0.035333,...,0,"(328000, 6956, 4.1, https://www.google.com//ma...",328000,6956.0,4.1,https://www.google.com//maps/place/Amsterdamse...,"(52.3140525, 4.9527714)",0.001698,0.105908,0.400000


In [206]:
final.to_csv("fulldata_draft4.csv", index=False)

In [203]:
miss1 = final[final["g_reviews"].isna()]

In [204]:
miss1

,name,label,wiki_page_obj,wiki_url,wiki_nl_views,wiki_norm_nl_views,wiki_eng_views,wiki_norm_eng_views,wiki_nr_of_edits,wiki_norm_nr_of_edits,...,Neighborhoodcombination,Neighborhood,UNESCO,Monuments,google,g_results,g_reviews,g_rating,g_link,g_coords
4,Amstelveld,Markt,<WikipediaPage 'Amstelveld'>,https://nl.wikipedia.org/wiki/Amstelveld,20987.0,0.041968,53163.0,5.664794e-03,47,0.030667,...,Grachtengordel-Zuid,Amstelveldbuurt,43,0,"(54800, nan, nan, https://www.google.com//maps...",54800,NaN,NaN,"https://www.google.com//maps/place/Amstelveld,...",NaN
13,Blauwbrug,Monument of Gebouw,<WikipediaPage 'Blauwbrug'>,https://nl.wikipedia.org/wiki/Blauwbrug,20136.0,0.040264,21425.0,2.282945e-03,72,0.047333,...,Nieuwmarkt/Lastage,Waterloopleinbuurt,11,8,"(124000, nan, 4.6, https://www.google.com//map...",124000,NaN,4.6,https://www.google.com//maps/dir//Blauwbrug+Br...,NaN
15,Dam,Monument of Gebouw,<WikipediaPage 'Dam (Amsterdam)'>,https://nl.wikipedia.org/wiki/Dam_(Amsterdam),120253.0,0.240687,329892.0,3.515171e-02,273,0.181333,...,Burgwallen-Nieuwe Zijde,Nieuwe Kerk e.o.,0,24,"(38800000, nan, nan, https://www.google.com//m...",38800000,NaN,NaN,"https://www.google.com//maps/place/Dam,+Amster...",NaN
21,De Wallen,Museum,<WikipediaPage 'De Wallen'>,https://nl.wikipedia.org/wiki/De_Wallen,241874.0,0.484158,1129722.0,1.203778e-01,346,0.230000,...,Burgwallen-Oude Zijde,Burgwallen Oost,0,111,"(4450000, nan, nan, https://www.google.com//ma...",4450000,NaN,NaN,"https://www.google.com//maps/place/De+Wallen,+...",NaN
31,Leidseplein,Monument of Gebouw,<WikipediaPage 'Leidseplein (Amsterdam)'>,https://nl.wikipedia.org/wiki/Leidseplein_(Ams...,40734.0,0.081499,95291.0,1.015375e-02,156,0.103333,...,De Weteringschans,Leidsebuurt Noordoost,10,26,"(2250000, nan, nan, https://www.google.com//ma...",2250000,NaN,NaN,https://www.google.com//maps/place/Leidseplein...,NaN
32,Lloyd Hotel,Monument of Gebouw,<WikipediaPage 'Lloyd Hotel (hotel)'>,https://nl.wikipedia.org/wiki/Lloyd_Hotel_(hotel),23296.0,0.046590,1928459.0,2.054873e-01,89,0.058667,...,Oostelijk Havengebied,Rietlanden,0,4,"(5600000, nan, 4.0, https://www.google.com//ma...",5600000,NaN,4.0,https://www.google.com//maps/place/Lloyd+Hotel...,"(52.3743324, 4.9348321)"
45,Noordermarkt,Markt,<WikipediaPage 'Noordermarkt'>,https://nl.wikipedia.org/wiki/Noordermarkt,18146.0,0.036280,25029.0,2.666970e-03,61,0.040000,...,Jordaan,Anjeliersbuurt Noord,48,6,"(234000, nan, nan, https://www.google.com//map...",234000,NaN,NaN,https://www.google.com//maps/place/Noordermark...,NaN
46,Olympisch Stadion,Monument of Gebouw,<WikipediaPage 'Olympisch Stadion (Amsterdam)'>,https://nl.wikipedia.org/wiki/Olympisch_Stadio...,119368.0,0.238915,215010.0,2.291043e-02,435,0.289333,...,Stadionbuurt,Olympisch Stadion e.o.,0,4,"(981000, nan, 4.4, nan, nan)",981000,NaN,4.4,NaN,NaN
55,Rembrandtplein,Monument of Gebouw,<WikipediaPage 'Rembrandtplein (Amsterdam)'>,https://nl.wikipedia.org/wiki/Rembrandtplein_(...,52276.0,0.104605,104845.0,1.117178e-02,159,0.105333,...,Grachtengordel-Zuid,Rembrandtpleinbuurt,28,41,"(590000, nan, nan, https://www.google.com//map...",590000,NaN,NaN,https://www.google.com//maps/place/Rembrandtpl...,NaN
56,Rembrandttoren,Monument of Gebouw,<WikipediaPage 'Rembrandttoren'>,https://nl.wikipedia.org/wiki/Rembrandttoren,60104.0,0.120275,50659.0,5.397980e-03,111,0.073333,...,Omval/Overamstel,De Omval,0,0,"(16500, nan, nan, https://www.google.com//maps...",16500,NaN,NaN,https://www.google.com//maps/place/Rembrandtto...,NaN


In [13]:
l = ['Amsterdam Dungeon', "Brouwerij 't IJ", 'Brouwerij De Prael', 'Dappermarkt', 'Openbare Bibliotheek Amsterdam', 'Stadhuis van Amsterdam', 'Uitmarkt', 'Elisabeth Otter-Knoll', 'Eigen Haard ', 'Sarphatimonument', 'Gerrit van der Veen', 'Museum Het Schip', 'Sint-Rosaschool', 'Vergulden Eenhoorn', 'Koninklijk Instituut voor de Tropen', 'Nederlandsche Dok en Scheepsbouw Maatschappij', 'Cornelis Schuytstraat', 'Lyceumbrug', 'De Dageraad ', 'De Bijenkorf ', 'Stadsarchief Amsterdam', 'ARCAM', 'Kromhout ', 'Tassenmuseum Amsterdam', 'Museum Vrolik', 'Amsterdam Pipe Museum', 'Diamantmuseum ', 'Theater De Engelenbak', 'Theater Frascati', 'Boom Chicago', 'Filmtheater Cinecenter']
df = pd.DataFrame(data={"name":l})

In [16]:
df1 = df[:10]
df2 = df[10:20]
df3 = df[20:]

,name
20,Stadsarchief Amsterdam
21,ARCAM
22,Kromhout
23,Tassenmuseum Amsterdam
24,Museum Vrolik
25,Amsterdam Pipe Museum
26,Diamantmuseum
27,Theater De Engelenbak
28,Theater Frascati
29,Boom Chicago


In [20]:
df1["google"] = df1["name"].apply(lambda a: scrape_google(a))

<ipython-input-20-0fda27a00163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["google"] = df1["name"].apply(lambda a: scrape_google(a))


In [21]:
df1

,name,google
0,Amsterdam Dungeon,"(7550000, 9595, 4.4, https://www.google.com//m..."
1,Brouwerij 't IJ,"(171000, 6659, 4.6, https://www.google.com//ma..."
2,Brouwerij De Prael,"(43500, nan, nan, https://www.google.com//maps..."
3,Dappermarkt,"(79900, nan, 5.0, nan, nan)"
4,Openbare Bibliotheek Amsterdam,"(5090000, nan, nan, https://www.google.com//ma..."
5,Stadhuis van Amsterdam,"(2270000, nan, 3.7, https://www.google.com//ma..."
6,Uitmarkt,"(110000, nan, nan, nan, nan)"
7,Elisabeth Otter-Knoll,"(392000, nan, 4.5, https://www.google.com//map..."
8,Eigen Haard,"(1660000, nan, 2.5, https://www.google.com//ma..."
9,Sarphatimonument,"(2110, nan, nan, https://www.google.com//maps/..."


In [22]:
df2["google"] = df2["name"].apply(lambda a: scrape_google(a))

<ipython-input-22-9db8ff4a2bd4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["google"] = df2["name"].apply(lambda a: scrape_google(a))


In [23]:
df2

,name,google
10,Gerrit van der Veen,"(560000, nan, nan, https://www.google.com//map..."
11,Museum Het Schip,"(1240000, nan, 4.5, https://www.google.com//ma..."
12,Sint-Rosaschool,"(500000, nan, nan, https://www.google.com//map..."
13,Vergulden Eenhoorn,"(19200, 1034, 4.3, https://www.google.com//map..."
14,Koninklijk Instituut voor de Tropen,"(126000, nan, 4.4, https://www.google.com//map..."
15,Nederlandsche Dok en Scheepsbouw Maatschappij,"(11500, nan, nan, https://www.google.com//maps..."
16,Cornelis Schuytstraat,"(93200, nan, nan, https://www.google.com//maps..."
17,Lyceumbrug,"(2740, nan, nan, https://www.google.com//maps/..."
18,De Dageraad,"(522000, nan, 4.4, https://www.google.com//map..."
19,De Bijenkorf,"(5050000, 13501, 4.3, https://www.google.com//..."


In [25]:
df3["google"] = df3["name"].apply(lambda a: scrape_google(a))

<ipython-input-25-4ee49c643c07>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["google"] = df3["name"].apply(lambda a: scrape_google(a))


In [26]:
df3

,name,google
20,Stadsarchief Amsterdam,"(2110000, nan, 4.4, https://www.google.com//ma..."
21,ARCAM,"(214000, nan, 4.3, https://www.google.com//map..."
22,Kromhout,"(296000, nan, 4.5, https://www.google.com//map..."
23,Tassenmuseum Amsterdam,"(40300, nan, 4.3, https://www.google.com//maps..."
24,Museum Vrolik,"(55500, nan, 4.7, https://www.google.com//maps..."
25,Amsterdam Pipe Museum,"(16800000, nan, 4.4, https://www.google.com//m..."
26,Diamantmuseum,"(26600, 1698, 3.8, https://www.google.com//map..."
27,Theater De Engelenbak,"(8980, nan, nan, https://www.google.com//maps/..."
28,Theater Frascati,"(431000, nan, 4.5, https://www.google.com//map..."
29,Boom Chicago,"(12300000, 1139, 4.4, https://www.google.com//..."


In [27]:
extra = pd.concat([df1,df2,df3])
extra

,name,google
0,Amsterdam Dungeon,"(7550000, 9595, 4.4, https://www.google.com//m..."
1,Brouwerij 't IJ,"(171000, 6659, 4.6, https://www.google.com//ma..."
2,Brouwerij De Prael,"(43500, nan, nan, https://www.google.com//maps..."
3,Dappermarkt,"(79900, nan, 5.0, nan, nan)"
4,Openbare Bibliotheek Amsterdam,"(5090000, nan, nan, https://www.google.com//ma..."
5,Stadhuis van Amsterdam,"(2270000, nan, 3.7, https://www.google.com//ma..."
6,Uitmarkt,"(110000, nan, nan, nan, nan)"
7,Elisabeth Otter-Knoll,"(392000, nan, 4.5, https://www.google.com//map..."
8,Eigen Haard,"(1660000, nan, 2.5, https://www.google.com//ma..."
9,Sarphatimonument,"(2110, nan, nan, https://www.google.com//maps/..."


In [28]:
extra["g_results"] = extra["google"].apply(lambda a: a[0])
extra["g_reviews"] = extra["google"].apply(lambda a: a[1])
extra["g_rating"] = extra["google"].apply(lambda a: a[2])
extra["g_link"] = extra["google"].apply(lambda a: a[3])
extra["g_coords"] = extra["google"].apply(lambda a: a[4])
extra

,name,google,g_results,g_reviews,g_rating,g_link,g_coords
0,Amsterdam Dungeon,"(7550000, 9595, 4.4, https://www.google.com//m...",7550000,9595.0,4.4,https://www.google.com//maps/place/Dungeon+Ams...,"(52.3701599, 4.8921108)"
1,Brouwerij 't IJ,"(171000, 6659, 4.6, https://www.google.com//ma...",171000,6659.0,4.6,https://www.google.com//maps/place/Brouwerij+'...,"(52.3666901, 4.9264241)"
2,Brouwerij De Prael,"(43500, nan, nan, https://www.google.com//maps...",43500,NaN,NaN,https://www.google.com//maps/dir//Brouwerij+De...,NaN
3,Dappermarkt,"(79900, nan, 5.0, nan, nan)",79900,NaN,5.0,NaN,NaN
4,Openbare Bibliotheek Amsterdam,"(5090000, nan, nan, https://www.google.com//ma...",5090000,NaN,NaN,https://www.google.com//maps/dir//OBA+Buitenve...,NaN
5,Stadhuis van Amsterdam,"(2270000, nan, 3.7, https://www.google.com//ma...",2270000,NaN,3.7,https://www.google.com//maps/dir//Gemeente+Ams...,NaN
6,Uitmarkt,"(110000, nan, nan, nan, nan)",110000,NaN,NaN,NaN,NaN
7,Elisabeth Otter-Knoll,"(392000, nan, 4.5, https://www.google.com//map...",392000,NaN,4.5,https://www.google.com//maps/place/Elisabeth+O...,"(52.3303056, 4.8788893)"
8,Eigen Haard,"(1660000, nan, 2.5, https://www.google.com//ma...",1660000,NaN,2.5,https://www.google.com//maps/place/Woningstich...,"(52.3857789, 4.8368075)"
9,Sarphatimonument,"(2110, nan, nan, https://www.google.com//maps/...",2110,NaN,NaN,https://www.google.com//maps/place/Sarphatimon...,"(52.3546165, 4.8938225)"


In [29]:
extra.isna().sum()/len(extra)

name         0.000000
google       0.000000
g_results    0.000000
g_reviews    0.806452
g_rating     0.322581
g_link       0.064516
g_coords     0.290323
dtype: float64

In [32]:
miss2 = extra[extra["g_reviews"].isna()]
len(miss2)

25

In [35]:
m1_ = miss2[:15]
m2_ = miss2[15:]

In [36]:
m1_["g_reviews"] = m1_["name"].apply(lambda a: get_reviews(a))

<ipython-input-36-6086b544a24a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m1_["g_reviews"] = m1_["name"].apply(lambda a: get_reviews(a))


In [37]:
m1_

,name,google,g_results,g_reviews,g_rating,g_link,g_coords
2,Brouwerij De Prael,"(43500, nan, nan, https://www.google.com//maps...",43500,NaN,NaN,https://www.google.com//maps/dir//Brouwerij+De...,NaN
3,Dappermarkt,"(79900, nan, 5.0, nan, nan)",79900,NaN,5.0,NaN,NaN
4,Openbare Bibliotheek Amsterdam,"(5090000, nan, nan, https://www.google.com//ma...",5090000,NaN,NaN,https://www.google.com//maps/dir//OBA+Buitenve...,NaN
5,Stadhuis van Amsterdam,"(2270000, nan, 3.7, https://www.google.com//ma...",2270000,NaN,3.7,https://www.google.com//maps/dir//Gemeente+Ams...,NaN
6,Uitmarkt,"(110000, nan, nan, nan, nan)",110000,NaN,NaN,NaN,NaN
7,Elisabeth Otter-Knoll,"(392000, nan, 4.5, https://www.google.com//map...",392000,2.0,4.5,https://www.google.com//maps/place/Elisabeth+O...,"(52.3303056, 4.8788893)"
8,Eigen Haard,"(1660000, nan, 2.5, https://www.google.com//ma...",1660000,218.0,2.5,https://www.google.com//maps/place/Woningstich...,"(52.3857789, 4.8368075)"
9,Sarphatimonument,"(2110, nan, nan, https://www.google.com//maps/...",2110,NaN,NaN,https://www.google.com//maps/place/Sarphatimon...,"(52.3546165, 4.8938225)"
10,Gerrit van der Veen,"(560000, nan, nan, https://www.google.com//map...",560000,35.0,NaN,https://www.google.com//maps/place/Gerrit+van+...,"(52.3486739, 4.874355)"
11,Museum Het Schip,"(1240000, nan, 4.5, https://www.google.com//ma...",1240000,759.0,4.5,https://www.google.com//maps/place/Museum+Het+...,"(52.3905672, 4.8737283)"


In [38]:
m2_["g_reviews"] = m2_["name"].apply(lambda a: get_reviews(a))

<ipython-input-38-806f23a671cd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m2_["g_reviews"] = m2_["name"].apply(lambda a: get_reviews(a))


In [39]:
m2_

,name,google,g_results,g_reviews,g_rating,g_link,g_coords
18,De Dageraad,"(522000, nan, 4.4, https://www.google.com//map...",522000,88.0,4.4,https://www.google.com//maps/place/Museum+De+D...,"(52.3498744, 4.8989827)"
20,Stadsarchief Amsterdam,"(2110000, nan, 4.4, https://www.google.com//ma...",2110000,NaN,4.4,https://www.google.com//maps/dir//Stadsarchief...,NaN
21,ARCAM,"(214000, nan, 4.3, https://www.google.com//map...",214000,189.0,4.3,https://www.google.com//maps/place/ARCAM/@52.3...,"(52.3714469, 4.9126544)"
22,Kromhout,"(296000, nan, 4.5, https://www.google.com//map...",296000,NaN,4.5,https://www.google.com//maps/dir//Museum+%E2%8...,NaN
23,Tassenmuseum Amsterdam,"(40300, nan, 4.3, https://www.google.com//maps...",40300,923.0,4.3,https://www.google.com//maps/place/Tassenmuseu...,"(52.3652211, 4.896893)"
24,Museum Vrolik,"(55500, nan, 4.7, https://www.google.com//maps...",55500,481.0,4.7,https://www.google.com//maps/place/Museum+Vrol...,"(52.2941667, 4.96)"
25,Amsterdam Pipe Museum,"(16800000, nan, 4.4, https://www.google.com//m...",16800000,342.0,4.4,https://www.google.com//maps/place/Amsterdam+P...,"(52.3641198, 4.8852857)"
27,Theater De Engelenbak,"(8980, nan, nan, https://www.google.com//maps/...",8980,NaN,NaN,https://www.google.com//maps/place/Stichting+%...,"(52.3697795, 4.8935809)"
28,Theater Frascati,"(431000, nan, 4.5, https://www.google.com//map...",431000,170.0,4.5,https://www.google.com//maps/place/Frascati/@5...,"(52.3703622, 4.8936327)"
30,Filmtheater Cinecenter,"(5310, nan, 4.2, https://www.google.com//maps/...",5310,516.0,4.2,https://www.google.com//maps/place/Cinecenter/...,"(52.364987, 4.881728)"


In [40]:
m_final2 = pd.concat([m1_,m2_])
m_final2

,name,google,g_results,g_reviews,g_rating,g_link,g_coords
2,Brouwerij De Prael,"(43500, nan, nan, https://www.google.com//maps...",43500,NaN,NaN,https://www.google.com//maps/dir//Brouwerij+De...,NaN
3,Dappermarkt,"(79900, nan, 5.0, nan, nan)",79900,NaN,5.0,NaN,NaN
4,Openbare Bibliotheek Amsterdam,"(5090000, nan, nan, https://www.google.com//ma...",5090000,NaN,NaN,https://www.google.com//maps/dir//OBA+Buitenve...,NaN
5,Stadhuis van Amsterdam,"(2270000, nan, 3.7, https://www.google.com//ma...",2270000,NaN,3.7,https://www.google.com//maps/dir//Gemeente+Ams...,NaN
6,Uitmarkt,"(110000, nan, nan, nan, nan)",110000,NaN,NaN,NaN,NaN
7,Elisabeth Otter-Knoll,"(392000, nan, 4.5, https://www.google.com//map...",392000,2.0,4.5,https://www.google.com//maps/place/Elisabeth+O...,"(52.3303056, 4.8788893)"
8,Eigen Haard,"(1660000, nan, 2.5, https://www.google.com//ma...",1660000,218.0,2.5,https://www.google.com//maps/place/Woningstich...,"(52.3857789, 4.8368075)"
9,Sarphatimonument,"(2110, nan, nan, https://www.google.com//maps/...",2110,NaN,NaN,https://www.google.com//maps/place/Sarphatimon...,"(52.3546165, 4.8938225)"
10,Gerrit van der Veen,"(560000, nan, nan, https://www.google.com//map...",560000,35.0,NaN,https://www.google.com//maps/place/Gerrit+van+...,"(52.3486739, 4.874355)"
11,Museum Het Schip,"(1240000, nan, 4.5, https://www.google.com//ma...",1240000,759.0,4.5,https://www.google.com//maps/place/Museum+Het+...,"(52.3905672, 4.8737283)"


In [41]:
for i in m_final2.index:
    extra.at[i,"g_reviews"] = m_final2.at[i,"g_reviews"]

In [42]:
extra.isna().sum()/len(extra)

name         0.000000
google       0.000000
g_results    0.000000
g_reviews    0.387097
g_rating     0.322581
g_link       0.064516
g_coords     0.290323
dtype: float64

In [43]:
extra["g_norm_results"] = (extra["g_results"]-extra["g_results"].min())/(extra["g_results"].max()-extra["g_results"].min())
extra["g_norm_reviews"] = (extra["g_reviews"]-extra["g_reviews"].min())/(extra["g_reviews"].max()-extra["g_reviews"].min())
extra["g_norm_rating"] = (extra["g_rating"]-extra["g_rating"].min())/(extra["g_rating"].max()-extra["g_rating"].min())

In [44]:
extra.to_csv("extra.csv", index=False)